In [2]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
print("mean length = ",(len([x for item in all_training_text for x in item]))/len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,8,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(mean(manhattanacc))
print(mean(cosineacc))
print(mean(euclideanacc))
print(mean(braycurtisacc))
print(mean(canberraacc))
print(mean(jaccardacc))
print(mean(yuleacc))


txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_chosen_set.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_chosen_set.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_chosen_set.csv 1028
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_chosen_set.csv 1092
length of features used = 2120
len(all_training_text) 152
mean length =  1252.296052631579
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
3
0.8028169014084507
0.7676056338028169
0.8028169014084507
0.795774647887324
0.6267605633802817
0.8380281690140845
0.7746478873239436
************************************ ****************************
5
0.7464788732394366
0.7605633802816901


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.7816901408450704
0.7746478873239436
0.6338028169014085
0.823943661971831
0.7464788732394366
************************************ ****************************
7
0.7183098591549296
0.7323943661971831
0.7253521126760564
0.7746478873239436
0.6197183098591549
0.7746478873239436
0.7323943661971831
************************************ ****************************
0.755868544600939
0.7535211267605634
0.7699530516431925
0.7816901408450704
0.6267605633802817
0.812206572769953
0.7511737089201878


In [3]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_chosen_set.csv"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,21,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(mean(manhattanacc))
print(mean(cosineacc))
print(mean(euclideanacc))
print(mean(braycurtisacc))
print(mean(canberraacc))
print(mean(jaccardacc))
print(mean(yuleacc))



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_chosen_set.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_chosen_set.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_chosen_set.csv 1028
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_chosen_set.csv 1092
length of features used = 2120
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
3
0.8028169014084507
0.7676056338028169
0.8028169014084507
0.795774647887324
0.6267605633802817
0.8380281690140845
0.7746478873239436
************************************ ****************************
5
0.7464788732394366
0.7605633802816901


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.7816901408450704
0.7746478873239436
0.6338028169014085
0.823943661971831
0.7464788732394366
************************************ ****************************
7
0.7183098591549296
0.7323943661971831
0.7253521126760564
0.7746478873239436
0.6197183098591549
0.7746478873239436
0.7323943661971831
************************************ ****************************
9
0.676056338028169
0.7253521126760564
0.704225352112676
0.7605633802816901
0.6338028169014085
0.7323943661971831
0.7253521126760564
************************************ ****************************
11
0.6619718309859155
0.6901408450704225
0.7183098591549296
0.7112676056338029
0.6338028169014085
0.7253521126760564
0.6830985915492958
************************************ ****************************
13
0.7112676056338029
0.6830985915492958
0.6971830985915493
0.676056338028169
0.6338028169014085
0.7605633802816901
0.676056338028169
************************************ ****************************
15
0.6971830985915493
0.66901408450704

In [5]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015OR.2F.txt"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:100]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,21,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(mean(manhattanacc))
print(mean(cosineacc))
print(mean(euclideanacc))
print(mean(braycurtisacc))
print(mean(canberraacc))
print(mean(jaccardacc))
print(mean(yuleacc))




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015OR.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015OR.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015OR.1M.txt 100
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015OR.2F.txt 100
length of features used = 200
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
3
0.5704225352112676
0.5
0.5704225352112676
0.5774647887323944
0.5704225352112676
0.5
0.5
************************************ ****************************
5
0.6126760563380281
0.49295774647887325
0.6126760563380281
0.6056338028169014
0.5915492957746479
0.49295774647887325
0.5
************************************ ****************************
7
0.5211267605633803
0.49295774647887325
0.528169014084507
0.528169014084507
0.5211267605633803
0.49295774647887325
0.5
*****

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.5
0.5140845070422535
0.5070422535211268
0.4859154929577465
0.49295774647887325
0.5
************************************ ****************************
17
0.5563380281690141
0.5
0.5422535211267606
0.5422535211267606
0.5422535211267606
0.5
0.5
************************************ ****************************
19
0.5563380281690141
0.5
0.5422535211267606
0.5633802816901409
0.5492957746478874
0.49295774647887325
0.5
************************************ ****************************
0.5500782472613459
0.4984350547730829
0.5453834115805947
0.543035993740219
0.5312989045383412
0.49921752738654146
0.5


In [6]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015OR.2F.txt"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:150]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,21,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(mean(manhattanacc))
print(mean(cosineacc))
print(mean(euclideanacc))
print(mean(braycurtisacc))
print(mean(canberraacc))
print(mean(jaccardacc))
print(mean(yuleacc))





txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015OR.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015OR.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015OR.1M.txt 150
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015OR.2F.txt 150
length of features used = 300
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
3
0.5704225352112676
0.5
0.5704225352112676
0.5774647887323944
0.5704225352112676
0.5
0.5
************************************ ****************************
5
0.6126760563380281
0.49295774647887325
0.6126760563380281
0.6056338028169014
0.5915492957746479
0.49295774647887325
0.5
************************************ ****************************
7
0.5211267605633803
0.49295774647887325
0.528169014084507
0.528169014084507
0.5211267605633803
0.49295774647887325
0.5
*****

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.5352112676056338
0.5352112676056338
0.49295774647887325
0.5
************************************ ****************************
15
0.5211267605633803
0.5
0.5140845070422535
0.5070422535211268
0.4859154929577465
0.49295774647887325
0.5
************************************ ****************************
17
0.5563380281690141
0.5
0.5422535211267606
0.5422535211267606
0.5422535211267606
0.5
0.5
************************************ ****************************
19
0.5563380281690141
0.5
0.5422535211267606
0.5633802816901409
0.5492957746478874
0.49295774647887325
0.5
************************************ ****************************
0.5500782472613459
0.4984350547730829
0.5453834115805947
0.543035993740219
0.5312989045383412
0.49921752738654146
0.5


In [7]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015OR.2F.txt"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:250]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,21,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(mean(manhattanacc))
print(mean(cosineacc))
print(mean(euclideanacc))
print(mean(braycurtisacc))
print(mean(canberraacc))
print(mean(jaccardacc))
print(mean(yuleacc))





txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015OR.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015OR.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015OR.1M.txt 250
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015OR.2F.txt 250
length of features used = 500
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
3
0.5422535211267606
0.5
0.5422535211267606
0.5492957746478874
0.5422535211267606
0.5
0.5
************************************ ****************************
5
0.5845070422535211
0.49295774647887325
0.5845070422535211
0.5774647887323944
0.5633802816901409
0.5
0.5
************************************ ****************************
7
0.5211267605633803
0.49295774647887325
0.528169014084507
0.528169014084507
0.5211267605633803
0.5
0.5
************************************ 

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.5
0.4859154929577465
0.47183098591549294
0.5
0.5
************************************ ****************************
11
0.5563380281690141
0.5
0.5492957746478874
0.5492957746478874
0.5140845070422535
0.5
0.5
************************************ ****************************
13
0.5563380281690141
0.5
0.5563380281690141
0.5352112676056338
0.5352112676056338
0.5
0.5
************************************ ****************************
15
0.528169014084507
0.5
0.5211267605633803
0.5140845070422535
0.4859154929577465
0.5
0.5
************************************ ****************************
17
0.5563380281690141
0.5
0.5422535211267606
0.5422535211267606
0.5422535211267606
0.5
0.5
************************************ ****************************
19
0.5563380281690141
0.5
0.5422535211267606
0.5633802816901409
0.5492957746478874
0.5
0.5
************************************ ****************************
0.5453834115805947
0.4984350547730829
0.5406885758998435
0.5383411580594679
0.525039123630673
0.5
0

In [8]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015PMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015PMI.2F.txt"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:100]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,21,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(mean(manhattanacc))
print(mean(cosineacc))
print(mean(euclideanacc))
print(mean(braycurtisacc))
print(mean(canberraacc))
print(mean(jaccardacc))
print(mean(yuleacc))





txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015PMI.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015PMI.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015PMI.1M.txt 100
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015PMI.2F.txt 100
length of features used = 200
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
3
0.5704225352112676
0.5
0.5704225352112676
0.5774647887323944
0.5704225352112676
0.5
0.5
************************************ ****************************
5
0.6126760563380281
0.49295774647887325
0.6126760563380281
0.6056338028169014
0.5915492957746479
0.49295774647887325
0.5
************************************ ****************************
7
0.5211267605633803
0.49295774647887325
0.528169014084507
0.528169014084507
0.5211267605633803
0.49295774647887325
0.5
*

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.5070422535211268
0.4859154929577465
0.49295774647887325
0.5
************************************ ****************************
17
0.5563380281690141
0.5
0.5422535211267606
0.5422535211267606
0.5422535211267606
0.5
0.5
************************************ ****************************
19
0.5563380281690141
0.5
0.5422535211267606
0.5633802816901409
0.5492957746478874
0.49295774647887325
0.5
************************************ ****************************
0.5500782472613459
0.4984350547730829
0.5453834115805947
0.543035993740219
0.5312989045383412
0.49921752738654146
0.5


In [9]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015PMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015PMI.2F.txt"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:150]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,21,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(mean(manhattanacc))
print(mean(cosineacc))
print(mean(euclideanacc))
print(mean(braycurtisacc))
print(mean(canberraacc))
print(mean(jaccardacc))
print(mean(yuleacc))






txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015PMI.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015PMI.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015PMI.1M.txt 150
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015PMI.2F.txt 150
length of features used = 300
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
3
0.5704225352112676
0.5
0.5704225352112676
0.5774647887323944
0.5704225352112676
0.5
0.5
************************************ ****************************
5
0.6126760563380281
0.49295774647887325
0.6126760563380281
0.6056338028169014
0.5915492957746479
0.49295774647887325
0.5
************************************ ****************************
7
0.5211267605633803
0.49295774647887325
0.528169014084507
0.528169014084507
0.5211267605633803
0.49295774647887325
0.5
*

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.5563380281690141
0.5
0.5563380281690141
0.5352112676056338
0.5352112676056338
0.49295774647887325
0.5
************************************ ****************************
15
0.5211267605633803
0.5
0.5140845070422535
0.5070422535211268
0.4859154929577465
0.49295774647887325
0.5
************************************ ****************************
17
0.5563380281690141
0.5
0.5422535211267606
0.5422535211267606
0.5422535211267606
0.5
0.5
************************************ ****************************
19
0.5563380281690141
0.5
0.5422535211267606
0.5633802816901409
0.5492957746478874
0.49295774647887325
0.5
************************************ ****************************
0.5500782472613459
0.4984350547730829
0.5453834115805947
0.543035993740219
0.5312989045383412
0.49921752738654146
0.5


In [10]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015PMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015PMI.2F.txt"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:250]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,21,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(mean(manhattanacc))
print(mean(cosineacc))
print(mean(euclideanacc))
print(mean(braycurtisacc))
print(mean(canberraacc))
print(mean(jaccardacc))
print(mean(yuleacc))






txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015PMI.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015PMI.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015PMI.1M.txt 250
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015PMI.2F.txt 250
length of features used = 500
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
3
0.5422535211267606
0.5
0.5422535211267606
0.5492957746478874
0.5422535211267606
0.5
0.5
************************************ ****************************
5
0.5845070422535211
0.49295774647887325
0.5845070422535211
0.5774647887323944
0.5633802816901409
0.5
0.5
************************************ ****************************
7
0.5211267605633803
0.49295774647887325
0.528169014084507
0.528169014084507
0.5211267605633803
0.5
0.5
*********************************

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.5
0.4859154929577465
0.47183098591549294
0.5
0.5
************************************ ****************************
11
0.5563380281690141
0.5
0.5492957746478874
0.5492957746478874
0.5140845070422535
0.5
0.5
************************************ ****************************
13
0.5563380281690141
0.5
0.5563380281690141
0.5352112676056338
0.5352112676056338
0.5
0.5
************************************ ****************************
15
0.528169014084507
0.5
0.5211267605633803
0.5140845070422535
0.4859154929577465
0.5
0.5
************************************ ****************************
17
0.5563380281690141
0.5
0.5422535211267606
0.5422535211267606
0.5422535211267606
0.5
0.5
************************************ ****************************
19
0.5563380281690141
0.5
0.5422535211267606
0.5633802816901409
0.5492957746478874
0.5
0.5
************************************ ****************************
0.5453834115805947
0.4984350547730829
0.5406885758998435
0.5383411580594679
0.525039123630673
0.5
0

In [11]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:100]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,21,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(mean(manhattanacc))
print(mean(cosineacc))
print(mean(euclideanacc))
print(mean(braycurtisacc))
print(mean(canberraacc))
print(mean(jaccardacc))
print(mean(yuleacc))






txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt 100
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt 100
length of features used = 200
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
3
0.45774647887323944
0.5
0.45774647887323944
0.4507042253521127
0.45774647887323944
0.5070422535211268
0.5
************************************ ****************************
5
0.528169014084507
0.5
0.528169014084507
0.528169014084507
0.5352112676056338
0.5070422535211268
0.5
************************************ ****************************
7
0.5
0.5
0.5
0.5
0.5
0.5
0.5
************************************ ****************************
9
0.478873239436619

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.5070422535211268
0.5
0.5
************************************ ****************************
17
0.4788732394366197
0.5
0.4788732394366197
0.5
0.47183098591549294
0.5
0.5
************************************ ****************************
19
0.5211267605633803
0.5
0.5352112676056338
0.5
0.5
0.5
0.5
************************************ ****************************
0.5
0.5
0.5046948356807511
0.49608763693270735
0.4953051643192488
0.5015649452269171
0.5


In [12]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:150]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,21,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(mean(manhattanacc))
print(mean(cosineacc))
print(mean(euclideanacc))
print(mean(braycurtisacc))
print(mean(canberraacc))
print(mean(jaccardacc))
print(mean(yuleacc))







txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt 150
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt 150
length of features used = 300
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
3
0.45774647887323944
0.5
0.45774647887323944
0.4507042253521127
0.45774647887323944
0.5070422535211268
0.5
************************************ ****************************
5
0.528169014084507
0.5
0.528169014084507
0.528169014084507
0.5352112676056338
0.5070422535211268
0.5
************************************ ****************************
7
0.5
0.5
0.5
0.5
0.5
0.5
0.5
************************************ ****************************
9
0.478873239436619

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.5
************************************ ****************************
13
0.5211267605633803
0.5
0.5211267605633803
0.5070422535211268
0.5070422535211268
0.5
0.5
************************************ ****************************
15
0.5070422535211268
0.5
0.5140845070422535
0.5
0.5070422535211268
0.5
0.5
************************************ ****************************
17
0.4788732394366197
0.5
0.4788732394366197
0.5
0.47183098591549294
0.5
0.5
************************************ ****************************
19
0.5211267605633803
0.5
0.5352112676056338
0.5
0.5
0.5
0.5
************************************ ****************************
0.5
0.5
0.5046948356807511
0.49608763693270735
0.4953051643192488
0.5015649452269171
0.5


In [13]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list1 =  ([(r.split(":"))[1] for r in f])[0:250]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,21,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(mean(manhattanacc))
print(mean(cosineacc))
print(mean(euclideanacc))
print(mean(braycurtisacc))
print(mean(canberraacc))
print(mean(jaccardacc))
print(mean(yuleacc))







txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt 250
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt 250
length of features used = 500
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
3
0.47183098591549294
0.5140845070422535
0.47183098591549294
0.4788732394366197
0.4859154929577465
0.5211267605633803
0.5
************************************ ****************************
5
0.5070422535211268
0.49295774647887325
0.5140845070422535
0.5140845070422535
0.49295774647887325
0.5070422535211268
0.5
************************************ ****************************
7
0.49295774647887325
0.49295774647887325
0.5070422535211268
0.49295774647887325


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.4647887323943662
0.5
0.5
************************************ ****************************
11
0.5070422535211268
0.5
0.5070422535211268
0.5070422535211268
0.5070422535211268
0.5
0.5
************************************ ****************************
13
0.528169014084507
0.5
0.5211267605633803
0.5211267605633803
0.5140845070422535
0.5
0.5
************************************ ****************************
15
0.49295774647887325
0.5
0.5070422535211268
0.4859154929577465
0.47183098591549294
0.5
0.5
************************************ ****************************
17
0.45774647887323944
0.5
0.4788732394366197
0.4647887323943662
0.4647887323943662
0.5
0.5
************************************ ****************************
19
0.5070422535211268
0.5
0.5352112676056338
0.5
0.4859154929577465
0.5
0.5
************************************ ****************************
0.49295774647887325
0.5
0.5023474178403755
0.49295774647887325
0.48513302034428796
0.5031298904538342
0.5


In [14]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015CHI.txt"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
        print(txt_file, len(word_list))
        #word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,21,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(mean(manhattanacc))
print(mean(cosineacc))
print(mean(euclideanacc))
print(mean(braycurtisacc))
print(mean(canberraacc))
print(mean(jaccardacc))
print(mean(yuleacc))







txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015CHI.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015CHI.txt 200
length of features used = 200
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
3
0.47183098591549294
0.5070422535211268
0.47183098591549294
0.4788732394366197
0.4859154929577465
0.5140845070422535
0.5
************************************ ****************************
5
0.5
0.49295774647887325
0.5070422535211268
0.5070422535211268
0.5
0.5070422535211268
0.5
************************************ ****************************
7
0.5
0.49295774647887325
0.5070422535211268
0.5070422535211268
0.4859154929577465
0.5
0.5
************************************ ****************************
9
0.47183098591549294
0.5
0.49295774647887325
0.4788732394366197
0.4647887323943662
0.5
0.5
************************************ *******************

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.5
************************************ ****************************
17
0.4647887323943662
0.5
0.4788732394366197
0.47183098591549294
0.4647887323943662
0.5
0.5
************************************ ****************************
19
0.5070422535211268
0.5
0.5352112676056338
0.5070422535211268
0.4788732394366197
0.5
0.5
************************************ ****************************
0.49608763693270735
0.49921752738654146
0.5054773082942097
0.49608763693270735
0.4843505477308294
0.5023474178403756
0.5


In [15]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015CHI.txt"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list =  ([(r.split(":"))[1] for r in f])[0:300]
        print(txt_file, len(word_list))
        #word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,21,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(mean(manhattanacc))
print(mean(cosineacc))
print(mean(euclideanacc))
print(mean(braycurtisacc))
print(mean(canberraacc))
print(mean(jaccardacc))
print(mean(yuleacc))








txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015CHI.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015CHI.txt 300
length of features used = 300
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
3
0.5070422535211268
0.38028169014084506
0.5140845070422535
0.5
0.5
0.5070422535211268
0.5
************************************ ****************************
5
0.528169014084507
0.5070422535211268
0.5140845070422535
0.5422535211267606
0.4859154929577465
0.5070422535211268
0.5
************************************ ****************************
7
0.5
0.5070422535211268
0.5
0.5
0.5140845070422535
0.5352112676056338
0.5
************************************ ****************************
9
0.44366197183098594
0.4859154929577465
0.47183098591549294
0.5492957746478874
0.5070422535211268
0.5352112676056338
0.5
************************************ ********

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.5352112676056338
0.5
************************************ ****************************
13
0.4788732394366197
0.5211267605633803
0.5070422535211268
0.5422535211267606
0.49295774647887325
0.5352112676056338
0.5
************************************ ****************************
15
0.4859154929577465
0.5070422535211268
0.5140845070422535
0.5492957746478874
0.5070422535211268
0.5352112676056338
0.5
************************************ ****************************
17
0.49295774647887325
0.5070422535211268
0.5
0.5070422535211268
0.5140845070422535
0.5352112676056338
0.5
************************************ ****************************
19
0.5352112676056338
0.47183098591549294
0.528169014084507
0.5140845070422535
0.528169014084507
0.5352112676056338
0.5
************************************ ****************************
0.49374021909233173
0.48826291079812206
0.5062597809076682
0.5312989045383412
0.5054773082942097
0.5289514866979655
0.5


In [16]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015CHI.txt"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list =  ([(r.split(":"))[1] for r in f])[0:500]
        print(txt_file, len(word_list))
        #word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,21,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(mean(manhattanacc))
print(mean(cosineacc))
print(mean(euclideanacc))
print(mean(braycurtisacc))
print(mean(canberraacc))
print(mean(jaccardacc))
print(mean(yuleacc))








txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015CHI.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015CHI.txt 500
length of features used = 500
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
3
0.5070422535211268
0.38028169014084506
0.5140845070422535
0.5
0.5
0.5070422535211268
0.5
************************************ ****************************
5
0.528169014084507
0.5070422535211268
0.5140845070422535
0.5422535211267606
0.4859154929577465
0.5070422535211268
0.5
************************************ ****************************
7
0.5
0.5070422535211268
0.5
0.5
0.5140845070422535
0.5352112676056338
0.5
************************************ ****************************
9
0.44366197183098594


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.4859154929577465
0.47183098591549294
0.5492957746478874
0.5070422535211268
0.5352112676056338
0.5
************************************ ****************************
11
0.47183098591549294
0.5070422535211268
0.5070422535211268
0.5774647887323944
0.5
0.5352112676056338
0.5
************************************ ****************************
13
0.4788732394366197
0.5211267605633803
0.5070422535211268
0.5422535211267606
0.49295774647887325
0.5352112676056338
0.5
************************************ ****************************
15
0.4859154929577465
0.5070422535211268
0.5140845070422535
0.5492957746478874
0.5070422535211268
0.5352112676056338
0.5
************************************ ****************************
17
0.49295774647887325
0.5070422535211268
0.5
0.5070422535211268
0.5140845070422535
0.5352112676056338
0.5
************************************ ****************************
19
0.5352112676056338
0.47183098591549294
0.528169014084507
0.5140845070422535
0.528169014084507
0.53521126760563

In [17]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
        print(txt_file, len(word_list))
        #word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,21,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(mean(manhattanacc))
print(mean(cosineacc))
print(mean(euclideanacc))
print(mean(braycurtisacc))
print(mean(canberraacc))
print(mean(jaccardacc))
print(mean(yuleacc))








txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt 200
length of features used = 200
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
3
0.5
0.5
0.5
0.5
0.5
0.5
0.5
************************************ ****************************
5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
************************************ ****************************
7
0.5
0.5
0.5
0.5
0.5
0.5
0.5
************************************ ****************************
9
0.5
0.5
0.5
0.5
0.5
0.5
0.5
************************************ ****************************
11
0.5
0.5
0.5
0.5
0.5
0.5
0.5
************************************ ****************************
13
0.5


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.5
0.5
0.5
0.5
0.5
0.5
************************************ ****************************
15
0.5
0.5
0.5
0.5
0.5
0.5
0.5
************************************ ****************************
17
0.5
0.5
0.5
0.5
0.5
0.5
0.5
************************************ ****************************
19
0.5
0.5
0.5
0.5
0.5
0.5
0.5
************************************ ****************************
0.5
0.5
0.5
0.5
0.5
0.5
0.5


In [18]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list =  ([(r.split(":"))[1] for r in f])[0:300]
        print(txt_file, len(word_list))
        #word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,21,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(mean(manhattanacc))
print(mean(cosineacc))
print(mean(euclideanacc))
print(mean(braycurtisacc))
print(mean(canberraacc))
print(mean(jaccardacc))
print(mean(yuleacc))









txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt 300
length of features used = 300
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
3
0.5704225352112676
0.5
0.5704225352112676
0.5774647887323944
0.5774647887323944
0.5
0.5
************************************ ****************************
5
0.6126760563380281
0.5
0.6126760563380281
0.6056338028169014
0.6056338028169014
0.5
0.5
************************************ ****************************
7
0.5492957746478874
0.5
0.5492957746478874
0.5352112676056338
0.5352112676056338
0.5
0.5
************************************ ****************************
9
0.5070422535211268
0.5
0.5070422535211268
0.49295774647887325
0.49295774647887325
0.5
0.5
************************************ ****************************
11
0.5422535211267606
0.

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.5
************************************ ****************************
13
0.5704225352112676
0.5
0.5704225352112676
0.5492957746478874
0.5492957746478874
0.5
0.5
************************************ ****************************
15
0.5140845070422535
0.5
0.5140845070422535
0.5140845070422535
0.5140845070422535
0.5
0.5
************************************ ****************************
17
0.5422535211267606
0.5
0.5422535211267606
0.5211267605633803
0.5211267605633803
0.5
0.5
************************************ ****************************
19
0.5422535211267606
0.5
0.5422535211267606
0.5704225352112676
0.5704225352112676
0.5
0.5
************************************ ****************************
0.5500782472613459
0.5
0.5500782472613459
0.5461658841940532
0.5461658841940532
0.5
0.5


In [19]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list =  ([(r.split(":"))[1] for r in f])[0:500]
        print(txt_file, len(word_list))
        #word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=1000)")
##selector = SelectKBest(score_func=chi2, k=1000)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,21,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(mean(manhattanacc))
print(mean(cosineacc))
print(mean(euclideanacc))
print(mean(braycurtisacc))
print(mean(canberraacc))
print(mean(jaccardacc))
print(mean(yuleacc))









txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt 500
length of features used = 500
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
3
0.5704225352112676
0.5
0.5704225352112676
0.5774647887323944
0.5774647887323944
0.5
0.5
************************************ ****************************
5
0.6126760563380281
0.5
0.6126760563380281
0.6056338028169014
0.6056338028169014
0.5
0.5
************************************ ****************************
7
0.5492957746478874
0.5
0.5492957746478874
0.5352112676056338
0.5352112676056338


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.5
0.5
************************************ ****************************
9
0.5070422535211268
0.5
0.5070422535211268
0.49295774647887325
0.49295774647887325
0.5
0.5
************************************ ****************************
11
0.5422535211267606
0.5
0.5422535211267606
0.5492957746478874
0.5492957746478874
0.5
0.5
************************************ ****************************
13
0.5704225352112676
0.5
0.5704225352112676
0.5492957746478874
0.5492957746478874
0.5
0.5
************************************ ****************************
15
0.5140845070422535
0.5
0.5140845070422535
0.5140845070422535
0.5140845070422535
0.5
0.5
************************************ ****************************
17
0.5422535211267606
0.5
0.5422535211267606
0.5211267605633803
0.5211267605633803
0.5
0.5
************************************ ****************************
19
0.5422535211267606
0.5
0.5422535211267606
0.5704225352112676
0.5704225352112676
0.5
0.5
************************************ ************

In [66]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_train.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_train.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_test.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_test.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_chosen_set.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_chosen_set.csv"]
#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=chi2, k=300)")
selector = SelectKBest(score_func=mutual_info_classif, k=100)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
x =(dict(zip(selector.scores_, word_list)))
sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
print(sorted_x)
df = pd.DataFrame({'sorted_tokens' : sorted_x})
df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
print(len(fit.scores_))

#print(selector.scores_)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,21,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(mean(manhattanacc))
print(mean(cosineacc))
print(mean(euclideanacc))
print(mean(braycurtisacc))
print(mean(canberraacc))
print(mean(jaccardacc))
print(mean(yuleacc))









txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_chosen_set.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_chosen_set.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_female_chosen_set.csv 1028
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/PAN2015_tweet_male_chosen_set.csv 1092
length of features used = 2120
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
selector = SelectKBest(score_func=chi2, k=300)
[(0.18490316254752193, 'register'), (0.1563234597244998, 'love'), (0.134860113491315, 'hit'), (0.12852280063692345, 'happy'), (0.12247095774297523, 'publish'), (0.11267198478610752, 'v'), (0.11164728830480564, 'wish'), (0.11063426019130085, 'smile'), (0.11026734178548936, '&'), (0.11020601143197628, 'study'), (0.10929751994405867, 'draw'), (0.10770264856677136, 'ni

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.704225352112676
0.7253521126760564
0.6056338028169014
0.676056338028169
0.5915492957746479
************************************ ****************************
19
0.7253521126760564
0.7323943661971831
0.7183098591549296
0.6830985915492958
0.5985915492957746
0.6971830985915493
0.6126760563380281
************************************ ****************************
0.7112676056338029
0.729264475743349
0.7175273865414711
0.7245696400625978
0.6322378716744914
0.6549295774647887
0.5970266040688575


In [30]:
import unicodedata
def tokenizeLineUTF8(newLine):
   aListChar = []
   aListCode = []
   for i, c in enumerate(newLine):
      aListCode.append(unicodedata.category(c)[0])
      aListChar.append(c)
   seqChar = seqPunct = seqNumber = False
   aLen = len(aListCode)
   aToken = u''
   aListOfWord = []
   for anIndex in range(aLen):
      aCode = aListCode[anIndex]
      aChar = aListChar[anIndex]
      if ((aCode == "L") & (seqChar)):
         aToken = aToken + aChar
      elif ((aCode == "P") & ((seqPunct) | (seqNumber))):
         aToken = aToken + aChar
      elif ((aCode == "N") & ((seqPunct) | (seqNumber))):
         aToken = aToken + aChar
      elif ((aCode == "Z") | (aCode == "C") | (aCode == "M") | (aCode == "S")):
         if (len(aToken) > 0):
            aListOfWord.append(aToken)
         aToken = u''
         seqChar = seqPunct = seqNumber = False
      elif ((aCode == "L") & (not seqChar)):
         if (len(aToken) > 0):
            aListOfWord.append(aToken)
         aToken = aChar
         seqChar = True
         seqPunct = seqNumber = False
      elif ((aCode == "P") & (not seqPunct)):
         if (len(aToken) > 0):
            aListOfWord.append(aToken)
         aToken = aChar
         seqPunct = True
         seqChar = seqNumber = False
      elif ((aCode == "N") & (not seqNumber)):
         if (len(aToken) > 0):
            aListOfWord.append(aToken)
         aToken = aChar
         seqNumber = True
         seqChar = seqPunct = False
   if (len(aToken) > 0):
      aListOfWord.append(aToken)
   return(aListOfWord)

print(tokenizeLineUTF8("ennumerate, aListOfWord, append."))

['ennumerate', ',', 'aListOfWord', ',', 'append', '.']


In [67]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv"]
#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=300)")
##selector = SelectKBest(score_func=mutual_info_classif, k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##x =(dict(zip(selector.scores_, word_list)))
##sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
##print(sorted_x)
##df = pd.DataFrame({'sorted_tokens' : sorted_x})
##df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
##print(len(fit.scores_))

#print(selector.scores_)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,21,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(mean(manhattanacc))
print(mean(cosineacc))
print(mean(euclideanacc))
print(mean(braycurtisacc))
print(mean(canberraacc))
print(mean(jaccardacc))
print(mean(yuleacc))



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv 1063
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv 1109
length of features used = 2172
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
selector = SelectKBest(score_func=chi2, k=300)
[(0.19425441916459452, 'national'), (0.15430900458570873, 'recent'), (0.1507316968997141, 'higher'), (0.14531316287141527, ':)'), (0.13280391791146173, 'gimme'), (0.13118015789954374, 'announce'), (0.12495704986722522, 'pero'), (0.1231623689074901, 'share'), (0.11822027421597592, 'musical'), (0.11697759396405871, 'hero'), (0.1160

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.6971830985915493
0.704225352112676
0.6690140845070423
0.7183098591549296
************************************ ****************************
19
0.676056338028169
0.5845070422535211
0.6338028169014085
0.704225352112676
0.7253521126760564
0.6830985915492958
0.6690140845070423
************************************ ****************************
0.6768388106416275
0.6001564945226917
0.5672926447574335
0.6666666666666666
0.690923317683881
0.6697965571205008
0.6807511737089202


In [69]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv"]
#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=300)")
##selector = SelectKBest(score_func=mutual_info_classif, k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##x =(dict(zip(selector.scores_, word_list)))
##sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
##print(sorted_x)
##df = pd.DataFrame({'sorted_tokens' : sorted_x})
##df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
##print(len(fit.scores_))

#print(selector.scores_)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,15,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(round(mean(manhattanacc), 4))
print(round(mean(cosineacc), 4))
print(round(mean(euclideanacc), 4))
print(round(mean(braycurtisacc), 4))
print(round(mean(canberraacc), 4))
print(round(mean(jaccardacc), 4))
print(round(mean(yuleacc), 4))


txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv 1063
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv 1109
length of features used = 2172
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
3
0.7605633802816901
0.7746478873239436
0.7464788732394366
0.8028169014084507
0.6690140845070423
0.8169014084507042
0.7816901408450704
************************************ ****************************
5
0.7464788732394366
0.7535211267605634


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.704225352112676
0.795774647887324
0.647887323943662
0.8028169014084507
0.7394366197183099
************************************ ****************************
7
0.7394366197183099
0.7183098591549296
0.7253521126760564
0.7394366197183099
0.6549295774647887
0.7676056338028169
0.7464788732394366
************************************ ****************************
9
0.7464788732394366
0.7323943661971831
0.6971830985915493
0.7253521126760564
0.6549295774647887
0.7253521126760564
0.7183098591549296
************************************ ****************************
11
0.7535211267605634
0.7112676056338029
0.6901408450704225
0.676056338028169
0.6338028169014085
0.7323943661971831
0.6971830985915493
************************************ ****************************
13
0.7112676056338029
0.6971830985915493
0.6690140845070423
0.6408450704225352
0.6549295774647887
0.7253521126760564
0.6549295774647887
************************************ ****************************
0.743
0.7312
0.7054
0.73
0.6526
0.761

In [73]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv"]
#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
###def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=mutual_info_classif, k=200)")
selector = SelectKBest(score_func=mutual_info_classif, k=200)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
##x =(dict(zip(selector.scores_, word_list)))
##sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
##print(sorted_x)
##df = pd.DataFrame({'sorted_tokens' : sorted_x})
##df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
##print(len(fit.scores_))

#print(selector.scores_)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,15,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(round(mean(manhattanacc), 4))
print(round(mean(cosineacc), 4))
print(round(mean(euclideanacc), 4))
print(round(mean(braycurtisacc), 4))
print(round(mean(canberraacc), 4))
print(round(mean(jaccardacc), 4))
print(round(mean(yuleacc), 4))


txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv 1063
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv 1109
length of features used = 2172
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
selector = SelectKBest(score_func=mutual_info_classif, k=200)
3
0.7183098591549296
0.6408450704225352
0.6619718309859155
0.6690140845070423
0.7323943661971831
0.7253521126760564
0.7323943661971831
************************************ ****************************
5
0.6971830985915493
0.6197183098591549
0.647887323943662
0.7394366197183099
0.7394366197183099
0.7183098591549296


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.7394366197183099
0.6690140845070423
0.6901408450704225
************************************ ****************************
13
0.6619718309859155
0.5774647887323944
0.6338028169014085
0.7253521126760564
0.7464788732394366
0.6690140845070423
0.7394366197183099
************************************ ****************************
0.6854
0.6256
0.6373
0.7207
0.743
0.6948
0.7254


In [74]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv"]
#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
###def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=mutual_info_classif, k=300)")
selector = SelectKBest(score_func=mutual_info_classif, k=300)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
##x =(dict(zip(selector.scores_, word_list)))
##sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
##print(sorted_x)
##df = pd.DataFrame({'sorted_tokens' : sorted_x})
##df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
##print(len(fit.scores_))

#print(selector.scores_)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,15,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(round(mean(manhattanacc), 4))
print(round(mean(cosineacc), 4))
print(round(mean(euclideanacc), 4))
print(round(mean(braycurtisacc), 4))
print(round(mean(canberraacc), 4))
print(round(mean(jaccardacc), 4))
print(round(mean(yuleacc), 4))



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv 1063
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv 1109
length of features used = 2172
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
selector = SelectKBest(score_func=mutual_info_classif, k=300)
3
0.7253521126760564
0.6901408450704225
0.7112676056338029
0.7394366197183099
0.6830985915492958
0.7535211267605634
0.6971830985915493
************************************ ****************************
5
0.7183098591549296
0.7253521126760564
0.7183098591549296
0.7394366197183099
0.6408450704225352
0.7394366197183099

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.7112676056338029
0.6830985915492958
0.7323943661971831
0.6690140845070423
0.7323943661971831
0.704225352112676
************************************ ****************************
11
0.7323943661971831
0.704225352112676
0.6971830985915493
0.7394366197183099
0.6408450704225352
0.6830985915492958
0.7253521126760564
************************************ ****************************
13
0.7323943661971831
0.704225352112676
0.7253521126760564
0.7394366197183099
0.6549295774647887
0.6830985915492958
0.6901408450704225
************************************ ****************************
0.7359
0.7148
0.7019
0.7312
0.6573
0.7136
0.7019


In [75]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv"]
#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
###def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=mutual_info_classif, k=500)")
selector = SelectKBest(score_func=mutual_info_classif, k=500)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
##x =(dict(zip(selector.scores_, word_list)))
##sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
##print(sorted_x)
##df = pd.DataFrame({'sorted_tokens' : sorted_x})
##df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
##print(len(fit.scores_))

#print(selector.scores_)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,15,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(round(mean(manhattanacc), 4))
print(round(mean(cosineacc), 4))
print(round(mean(euclideanacc), 4))
print(round(mean(braycurtisacc), 4))
print(round(mean(canberraacc), 4))
print(round(mean(jaccardacc), 4))
print(round(mean(yuleacc), 4))

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv 1063
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv 1109
length of features used = 2172
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
selector = SelectKBest(score_func=mutual_info_classif, k=500)
3
0.7887323943661971
0.7605633802816901
0.7323943661971831
0.795774647887324
0.7112676056338029
0.7676056338028169
0.7323943661971831
************************************ ****************************
5
0.7887323943661971
0.6971830985915493
0.704225352112676
0.7887323943661971
0.704225352112676
0.7323943661971831
0.

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.7464788732394366
0.7464788732394366
0.7183098591549296
0.7887323943661971
0.6690140845070423
0.7253521126760564
0.6901408450704225
************************************ ****************************
9
0.7464788732394366
0.6971830985915493
0.7253521126760564
0.7605633802816901
0.6690140845070423
0.7464788732394366
0.7112676056338029
************************************ ****************************
11
0.7253521126760564
0.704225352112676
0.704225352112676
0.7535211267605634
0.6549295774647887
0.7183098591549296
0.7183098591549296
************************************ ****************************
13
0.7323943661971831
0.7253521126760564
0.6901408450704225
0.7605633802816901
0.6549295774647887
0.7323943661971831
0.7253521126760564
************************************ ****************************
0.7547
0.7218
0.7124
0.7746
0.6772
0.7371
0.7124


In [77]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv"]
#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
###def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=chi2, k=200)")
selector = SelectKBest(score_func=chi2, k=200)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
##x =(dict(zip(selector.scores_, word_list)))
##sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
##print(sorted_x)
##df = pd.DataFrame({'sorted_tokens' : sorted_x})
##df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
##print(len(fit.scores_))

#print(selector.scores_)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,15,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(round(mean(manhattanacc), 4))
print(round(mean(cosineacc), 4))
print(round(mean(euclideanacc), 4))
print(round(mean(braycurtisacc), 4))
print(round(mean(canberraacc), 4))
print(round(mean(jaccardacc), 4))
print(round(mean(yuleacc), 4))


txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv 1063
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv 1109
length of features used = 2172
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
selector = SelectKBest(score_func=chi2, k=200)
3
0.7464788732394366
0.7464788732394366
0.7394366197183099
0.7746478873239436
0.795774647887324
0.7676056338028169
0.795774647887324
************************************ ****************************
5
0.6830985915492958
0.7323943661971831
0.704225352112676
0.7535211267605634
0.7535211267605634
0.7605633802816901
0.788732394366197

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.6971830985915493
0.7464788732394366
0.7253521126760564
0.6690140845070423
0.6971830985915493
************************************ ****************************
13
0.7253521126760564
0.704225352112676
0.7183098591549296
0.7605633802816901
0.7253521126760564
0.6338028169014085
0.6830985915492958
************************************ ****************************
0.7136
0.73
0.7113
0.7547
0.7547
0.7066
0.7336


In [78]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv"]
#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
###def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=chi2, k=300)")
selector = SelectKBest(score_func=chi2, k=300)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
##x =(dict(zip(selector.scores_, word_list)))
##sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
##print(sorted_x)
##df = pd.DataFrame({'sorted_tokens' : sorted_x})
##df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
##print(len(fit.scores_))

#print(selector.scores_)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,15,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(round(mean(manhattanacc), 4))
print(round(mean(cosineacc), 4))
print(round(mean(euclideanacc), 4))
print(round(mean(braycurtisacc), 4))
print(round(mean(canberraacc), 4))
print(round(mean(jaccardacc), 4))
print(round(mean(yuleacc), 4))

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv 1063
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv 1109
length of features used = 2172
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
selector = SelectKBest(score_func=chi2, k=300)
3
0.7183098591549296
0.7535211267605634
0.7323943661971831
0.7535211267605634
0.795774647887324
0.8028169014084507
0.823943661971831
************************************ ****************************
5
0.7323943661971831
0.7323943661971831
0.7112676056338029
0.7112676056338029
0.7887323943661971
0.7605633802816901
0.76760563380281

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.7464788732394366
************************************ ****************************
9
0.7183098591549296
0.6971830985915493
0.7323943661971831
0.7323943661971831
0.7394366197183099
0.7816901408450704
0.8169014084507042
************************************ ****************************
11
0.7605633802816901
0.7112676056338029
0.6830985915492958
0.7253521126760564
0.7746478873239436
0.7605633802816901
0.8098591549295775
************************************ ****************************
13
0.7464788732394366
0.6971830985915493
0.704225352112676
0.7183098591549296
0.7816901408450704
0.7323943661971831
0.7535211267605634
************************************ ****************************
0.7312
0.723
0.7171
0.73
0.7758
0.7735
0.7864


In [79]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv"]
#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
###def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=chi2, k=500)")
selector = SelectKBest(score_func=chi2, k=500)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
##x =(dict(zip(selector.scores_, word_list)))
##sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
##print(sorted_x)
##df = pd.DataFrame({'sorted_tokens' : sorted_x})
##df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
##print(len(fit.scores_))

#print(selector.scores_)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,15,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(round(mean(manhattanacc), 4))
print(round(mean(cosineacc), 4))
print(round(mean(euclideanacc), 4))
print(round(mean(braycurtisacc), 4))
print(round(mean(canberraacc), 4))
print(round(mean(jaccardacc), 4))
print(round(mean(yuleacc), 4))

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv 1063
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv 1109
length of features used = 2172
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
selector = SelectKBest(score_func=chi2, k=500)
3
0.7394366197183099
0.7676056338028169
0.7323943661971831
0.7535211267605634
0.795774647887324
0.7816901408450704
0.7887323943661971
************************************ ****************************
5
0.6619718309859155
0.7183098591549296
0.6901408450704225
0.704225352112676
0.7816901408450704
0.7887323943661971


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.8098591549295775
************************************ ****************************
7
0.6619718309859155
0.647887323943662
0.6971830985915493
0.6830985915492958
0.7887323943661971
0.7887323943661971
0.7816901408450704
************************************ ****************************
9
0.6901408450704225
0.6408450704225352
0.6690140845070423
0.6901408450704225
0.8098591549295775
0.7464788732394366
0.7676056338028169
************************************ ****************************
11
0.704225352112676
0.647887323943662
0.676056338028169
0.6619718309859155
0.795774647887324
0.7887323943661971
0.7676056338028169
************************************ ****************************
13
0.704225352112676
0.6549295774647887
0.6408450704225352
0.6830985915492958
0.7746478873239436
0.7183098591549296
0.7323943661971831
************************************ ****************************
0.6937
0.6796
0.6843
0.696
0.7911
0.7688
0.7746


In [83]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_vocubulary.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_ vocubulary_set.1.csv"]
#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=300)")
##selector = SelectKBest(score_func=mutual_info_classif, k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##x =(dict(zip(selector.scores_, word_list)))
##sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
##print(sorted_x)
##df = pd.DataFrame({'sorted_tokens' : sorted_x})
##df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
##print(len(fit.scores_))

#print(selector.scores_)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,15,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(round(mean(manhattanacc), 4))
print(round(mean(cosineacc), 4))
print(round(mean(euclideanacc), 4))
print(round(mean(braycurtisacc), 4))
print(round(mean(canberraacc), 4))
print(round(mean(jaccardacc), 4))
print(round(mean(yuleacc), 4))



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_vocubulary.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_ vocubulary_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_vocubulary.1.csv 11167
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_ vocubulary_set.1.csv 11808
length of features used = 22975
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
3
0.8309859154929577
0.7887323943661971
0.7535211267605634
0.8309859154929577
0.6126760563380281
0.7394366197183099


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.6197183098591549
************************************ ****************************
5
0.795774647887324
0.7464788732394366
0.704225352112676
0.823943661971831
0.647887323943662
0.7464788732394366
0.6197183098591549
************************************ ****************************
7
0.7887323943661971
0.7253521126760564
0.7183098591549296
0.7394366197183099
0.6408450704225352
0.6971830985915493
0.5845070422535211
************************************ ****************************
9
0.7394366197183099
0.7183098591549296
0.6690140845070423
0.7253521126760564
0.6267605633802817
0.676056338028169
0.5845070422535211
************************************ ****************************
11
0.6971830985915493
0.7112676056338029
0.7253521126760564
0.7323943661971831
0.6408450704225352
0.6549295774647887
0.5774647887323944
************************************ ****************************
13
0.6267605633802817
0.704225352112676
0.6971830985915493
0.6619718309859155
0.6690140845070423
0.6267605633802817

In [4]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv"]
#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
print("mean length = ",(len([x for item in all_training_text for x in item]))/len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
###def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=200)")
selector = SelectKBest(score_func=f_classif, k=200)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
##x =(dict(zip(selector.scores_, word_list)))
##sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
##print(sorted_x)
##df = pd.DataFrame({'sorted_tokens' : sorted_x})
##df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
##print(len(fit.scores_))

#print(selector.scores_)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,15,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(round(mean(manhattanacc), 4))
print(round(mean(cosineacc), 4))
print(round(mean(euclideanacc), 4))
print(round(mean(braycurtisacc), 4))
print(round(mean(canberraacc), 4))
print(round(mean(jaccardacc), 4))
print(round(mean(yuleacc), 4))

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv 1063
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv 1109
length of features used = 2172
len(all_training_text) 152
mean length =  1424.9802631578948
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
selector = SelectKBest(score_func=f_classif, k=200)
3
0.7394366197183099
0.7605633802816901
0.7605633802816901
0.823943661971831
0.7253521126760564
0.6971830985915493
0.6830985915492958
************************************ ****************************
5
0.7464788732394366
0.7323943661971831
0.7605633802816901
0.823943661971831
0.7323943661971

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.7183098591549296
0.7112676056338029
0.6901408450704225
************************************ ****************************
11
0.6830985915492958
0.7253521126760564
0.7323943661971831
0.7323943661971831
0.7464788732394366
0.7253521126760564
0.6971830985915493
************************************ ****************************
13
0.6830985915492958
0.704225352112676
0.7394366197183099
0.7183098591549296
0.7605633802816901
0.704225352112676
0.6971830985915493
************************************ ****************************
0.7254
0.7277
0.7523
0.7805
0.7347
0.7124
0.6925


In [85]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv"]
#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
###def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=300)")
selector = SelectKBest(score_func=f_classif, k=300)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
##x =(dict(zip(selector.scores_, word_list)))
##sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
##print(sorted_x)
##df = pd.DataFrame({'sorted_tokens' : sorted_x})
##df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
##print(len(fit.scores_))

#print(selector.scores_)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,15,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(round(mean(manhattanacc), 4))
print(round(mean(cosineacc), 4))
print(round(mean(euclideanacc), 4))
print(round(mean(braycurtisacc), 4))
print(round(mean(canberraacc), 4))
print(round(mean(jaccardacc), 4))
print(round(mean(yuleacc), 4))

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv 1063
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv 1109
length of features used = 2172
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
selector = SelectKBest(score_func=f_classif, k=300)
3
0.7676056338028169
0.7112676056338029
0.7464788732394366
0.7464788732394366
0.7676056338028169
0.7323943661971831
0.7464788732394366
************************************ ****************************
5
0.7676056338028169
0.6971830985915493
0.7676056338028169
0.7816901408450704
0.7816901408450704
0.7183098591549296
0.7253521

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.7464788732394366
0.7605633802816901
0.7746478873239436
0.7535211267605634
0.7816901408450704
0.7183098591549296
0.7394366197183099
************************************ ****************************
11
0.6971830985915493
0.7394366197183099
0.7464788732394366
0.7746478873239436
0.7887323943661971
0.704225352112676
0.7112676056338029
************************************ ****************************
13
0.704225352112676
0.7323943661971831
0.7605633802816901
0.7535211267605634
0.7605633802816901
0.7323943661971831
0.7112676056338029
************************************ ****************************
0.7324
0.7359
0.7535
0.7617
0.7676
0.7183
0.7289


In [86]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv"]
#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
###def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=500)")
selector = SelectKBest(score_func=f_classif, k=500)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
##x =(dict(zip(selector.scores_, word_list)))
##sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
##print(sorted_x)
##df = pd.DataFrame({'sorted_tokens' : sorted_x})
##df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
##print(len(fit.scores_))

#print(selector.scores_)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,15,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(round(mean(manhattanacc), 4))
print(round(mean(cosineacc), 4))
print(round(mean(euclideanacc), 4))
print(round(mean(braycurtisacc), 4))
print(round(mean(canberraacc), 4))
print(round(mean(jaccardacc), 4))
print(round(mean(yuleacc), 4))

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv 1063
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv 1109
length of features used = 2172
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
selector = SelectKBest(score_func=f_classif, k=500)
3
0.7605633802816901
0.7605633802816901
0.7183098591549296
0.7746478873239436
0.704225352112676
0.7323943661971831
0.7746478873239436
************************************ ****************************
5
0.7676056338028169
0.7394366197183099
0.7605633802816901
0.7676056338028169
0.7253521126760564
0.7816901408450704


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.7746478873239436
************************************ ****************************
7
0.7464788732394366
0.7464788732394366
0.7253521126760564
0.7746478873239436
0.6971830985915493
0.7676056338028169
0.7535211267605634
************************************ ****************************
9
0.7253521126760564
0.7676056338028169
0.7253521126760564
0.7394366197183099
0.6901408450704225
0.7112676056338029
0.7394366197183099
************************************ ****************************
11
0.6619718309859155
0.7323943661971831
0.7323943661971831
0.7464788732394366
0.6830985915492958
0.704225352112676
0.7394366197183099
************************************ ****************************
13
0.6408450704225352
0.676056338028169
0.7323943661971831
0.7183098591549296
0.6690140845070423
0.704225352112676
0.7183098591549296
************************************ ****************************
0.7171
0.7371
0.7324
0.7535
0.6948
0.7336
0.75


In [87]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv"]
#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
###def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=200)")
selector = SelectKBest(score_func=f_classif, k=200)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
##x =(dict(zip(selector.scores_, word_list)))
##sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
##print(sorted_x)
##df = pd.DataFrame({'sorted_tokens' : sorted_x})
##df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
##print(len(fit.scores_))

#print(selector.scores_)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,15,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "rogerstanimoto")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        print("rogerstanimoto K-Nearest Neighbours")
        print(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(round(mean(manhattanacc), 4))
print(round(mean(cosineacc), 4))
print(round(mean(euclideanacc), 4))
print(round(mean(braycurtisacc), 4))
print(round(mean(canberraacc), 4))
print(round(mean(jaccardacc), 4))
print(round(mean(yuleacc), 4))

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv 1063
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv 1109
length of features used = 2172
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
selector = SelectKBest(score_func=f_classif, k=200)
3
0.7394366197183099
0.7605633802816901
0.7605633802816901
0.823943661971831
0.7253521126760564
0.6971830985915493
0.6830985915492958
rogerstanimoto K-Nearest Neighbours
0.7323943661971831
************************************ ****************************
5
0.7464788732394366
0.7323943661971831
0.7605633802816901
0.8239436619

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


rogerstanimoto K-Nearest Neighbours
0.7605633802816901
************************************ ****************************
9
0.7394366197183099
0.7253521126760564
0.7676056338028169
0.7816901408450704
0.7183098591549296
0.7112676056338029
0.6901408450704225
rogerstanimoto K-Nearest Neighbours
0.7676056338028169
************************************ ****************************
11
0.6830985915492958
0.7253521126760564
0.7323943661971831
0.7323943661971831
0.7464788732394366
0.7253521126760564
0.6971830985915493
rogerstanimoto K-Nearest Neighbours
0.7816901408450704
************************************ ****************************
13
0.6830985915492958
0.704225352112676
0.7394366197183099
0.7183098591549296
0.7605633802816901
0.704225352112676
0.6971830985915493
rogerstanimoto K-Nearest Neighbours
0.7676056338028169
************************************ ****************************
0.7254
0.7277
0.7523
0.7805
0.7347
0.7124
0.6925


In [88]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_vocubulary.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_ vocubulary_set.1.csv"]
#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=300)")
##selector = SelectKBest(score_func=mutual_info_classif, k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##x =(dict(zip(selector.scores_, word_list)))
##sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
##print(sorted_x)
##df = pd.DataFrame({'sorted_tokens' : sorted_x})
##df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
##print(len(fit.scores_))

#print(selector.scores_)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,15,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "rogerstanimoto")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        print("rogerstanimoto K-Nearest Neighbours")
        print(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(round(mean(manhattanacc), 4))
print(round(mean(cosineacc), 4))
print(round(mean(euclideanacc), 4))
print(round(mean(braycurtisacc), 4))
print(round(mean(canberraacc), 4))
print(round(mean(jaccardacc), 4))
print(round(mean(yuleacc), 4))


txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_vocubulary.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_ vocubulary_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_vocubulary.1.csv 11167
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_ vocubulary_set.1.csv 11808
length of features used = 22975
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
3
0.8309859154929577
0.7887323943661971
0.7535211267605634
0.8309859154929577
0.6126760563380281
0.7394366197183099


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.6197183098591549
rogerstanimoto K-Nearest Neighbours
0.5704225352112676
************************************ ****************************
5
0.795774647887324
0.7464788732394366
0.704225352112676
0.823943661971831
0.647887323943662
0.7464788732394366
0.6197183098591549
rogerstanimoto K-Nearest Neighbours
0.6690140845070423
************************************ ****************************
7
0.7887323943661971
0.7253521126760564
0.7183098591549296
0.7394366197183099
0.6408450704225352
0.6971830985915493
0.5845070422535211
rogerstanimoto K-Nearest Neighbours
0.6408450704225352
************************************ ****************************
9
0.7394366197183099
0.7183098591549296
0.6690140845070423
0.7253521126760564
0.6267605633802817
0.676056338028169
0.5845070422535211
rogerstanimoto K-Nearest Neighbours
0.647887323943662
************************************ ****************************
11
0.6971830985915493
0.7112676056338029
0.7253521126760564
0.7323943661971831
0.6408450704225352


In [89]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv"]
#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=300)")
##selector = SelectKBest(score_func=mutual_info_classif, k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##x =(dict(zip(selector.scores_, word_list)))
##sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
##print(sorted_x)
##df = pd.DataFrame({'sorted_tokens' : sorted_x})
##df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
##print(len(fit.scores_))

#print(selector.scores_)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)
def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator
"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,15,2):
##        acc=[]
        print(k)
    
        """
        8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        print("tanimoto_distance K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "rogerstanimoto")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        print("rogerstanimoto K-Nearest Neighbours")
        print(metrics.accuracy_score(y_test, y_pred))
        
        
        
        print("************************************ ****************************")
print(round(mean(manhattanacc), 4))
print(round(mean(cosineacc), 4))
print(round(mean(euclideanacc), 4))
print(round(mean(braycurtisacc), 4))
print(round(mean(canberraacc), 4))
print(round(mean(jaccardacc), 4))
print(round(mean(yuleacc), 4))



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv 1063
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv 1109
length of features used = 2172
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
3
0.7605633802816901
0.7746478873239436
0.7464788732394366
0.8028169014084507
0.6690140845070423
0.8169014084507042
0.7816901408450704
rogerstanimoto K-Nearest Neighbours
0.676056338028169
************************************ ****************************
5


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.7464788732394366
0.7535211267605634
0.704225352112676
0.795774647887324
0.647887323943662
0.8028169014084507
0.7394366197183099
rogerstanimoto K-Nearest Neighbours
0.6690140845070423
************************************ ****************************
7
0.7394366197183099
0.7183098591549296
0.7253521126760564
0.7394366197183099
0.6549295774647887
0.7676056338028169
0.7464788732394366
rogerstanimoto K-Nearest Neighbours
0.704225352112676
************************************ ****************************
9
0.7464788732394366
0.7323943661971831
0.6971830985915493
0.7253521126760564
0.6549295774647887
0.7253521126760564
0.7183098591549296
rogerstanimoto K-Nearest Neighbours
0.6901408450704225
************************************ ****************************
11
0.7535211267605634
0.7112676056338029
0.6901408450704225
0.676056338028169
0.6338028169014085
0.7323943661971831
0.6971830985915493
rogerstanimoto K-Nearest Neighbours
0.6690140845070423
************************************ ***********

In [90]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv"]
#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=300)")
##selector = SelectKBest(score_func=mutual_info_classif, k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##x =(dict(zip(selector.scores_, word_list)))
##sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
##print(sorted_x)
##df = pd.DataFrame({'sorted_tokens' : sorted_x})
##df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
##print(len(fit.scores_))

#print(selector.scores_)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)
def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator
"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,15,2):
##        acc=[]
        print(k)
    
        """
        8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        print("tanimoto_distance K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "rogerstanimoto")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        print("rogerstanimoto K-Nearest Neighbours")
        print(metrics.accuracy_score(y_test, y_pred))
        
        
        
        print("************************************ ****************************")
print(round(mean(manhattanacc), 4))
print(round(mean(cosineacc), 4))
print(round(mean(euclideanacc), 4))
print(round(mean(braycurtisacc), 4))
print(round(mean(canberraacc), 4))
print(round(mean(jaccardacc), 4))
print(round(mean(yuleacc), 4))




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv 1063
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv 1109
length of features used = 2172
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
3
tanimoto_distance K-Nearest Neighbours ['M' 'M' 'M' 'F' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'F' 'M'
 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'F' 'M'
 'M' 'M' 'M' 'M' 'M' 'M' 'F' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'F' 'M'
 'M' 'M' 'M' 'M' 'F' 'F' 'M' 'M' 'M' 'M' 'F' 'M' 'M' 'M' 'F' 'M' 'M' 'F'
 'F' 'F' 'F' 'M' 'F' 'F' 'M' 'M' 'F' 'M' 'F' 

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.7816901408450704
rogerstanimoto K-Nearest Neighbours
0.676056338028169
************************************ ****************************
5
tanimoto_distance K-Nearest Neighbours ['M' 'M' 'M' 'F' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'F' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'F' 'M'
 'M' 'M' 'M' 'M' 'M' 'M' 'F' 'M' 'F' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'F' 'M'
 'M' 'M' 'M' 'M' 'F' 'M' 'M' 'F' 'M' 'M' 'F' 'M' 'M' 'M' 'F' 'F' 'M' 'F'
 'F' 'M' 'F' 'F' 'M' 'F' 'M' 'M' 'F' 'F' 'F' 'M' 'F' 'F' 'M' 'F' 'M' 'F'
 'F' 'F' 'F' 'M' 'F' 'F' 'M' 'F' 'F' 'F' 'F' 'F' 'F' 'F' 'F' 'F' 'F' 'F'
 'F' 'F' 'F' 'M' 'F' 'F' 'F' 'M' 'F' 'M' 'F' 'F' 'F' 'F' 'F' 'M' 'F' 'M'
 'M' 'F' 'F' 'F' 'F' 'F' 'M' 'M' 'F' 'F' 'F' 'F' 'M' 'F' 'F' 'F']
0.795774647887324
0.7464788732394366
0.7535211267605634
0.704225352112676
0.795774647887324
0.647887323943662
0.8028169014084507
0.7394366197183099
rogerstanimoto K-Nearest Neighbours
0.6690140845070423
************************************ ***

In [91]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_vocubulary.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_ vocubulary_set.1.csv"]
#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
##print("selector = SelectKBest(score_func=chi2, k=300)")
##selector = SelectKBest(score_func=mutual_info_classif, k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##x =(dict(zip(selector.scores_, word_list)))
##sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
##print(sorted_x)
##df = pd.DataFrame({'sorted_tokens' : sorted_x})
##df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
##print(len(fit.scores_))

#print(selector.scores_)
##test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train =train_selected_features 
##X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)
def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator
    
        

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,15,2):
##        acc=[]
        print(k)
        """
        8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        print("tanimoto_distance K-Nearest Neighbours")
        print(metrics.accuracy_score(y_test, y_pred))
        


        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "rogerstanimoto")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        print("rogerstanimoto K-Nearest Neighbours")
        print(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(round(mean(manhattanacc), 4))
print(round(mean(cosineacc), 4))
print(round(mean(euclideanacc), 4))
print(round(mean(braycurtisacc), 4))
print(round(mean(canberraacc), 4))
print(round(mean(jaccardacc), 4))
print(round(mean(yuleacc), 4))



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_vocubulary.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_ vocubulary_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_vocubulary.1.csv 11167
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_ vocubulary_set.1.csv 11808
length of features used = 22975
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
3
tanimoto_distance K-Nearest Neighbours
0.8309859154929577
0.8309859154929577
0.7887323943661971
0.7535211267605634
0.8309859154929577
0.6126760563380281
0.7394366197183099


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


0.6197183098591549
rogerstanimoto K-Nearest Neighbours
0.5704225352112676
************************************ ****************************
5
tanimoto_distance K-Nearest Neighbours
0.823943661971831
0.795774647887324
0.7464788732394366
0.704225352112676
0.823943661971831
0.647887323943662
0.7464788732394366
0.6197183098591549
rogerstanimoto K-Nearest Neighbours
0.6690140845070423
************************************ ****************************
7
tanimoto_distance K-Nearest Neighbours
0.7394366197183099
0.7887323943661971
0.7253521126760564
0.7183098591549296
0.7394366197183099
0.6408450704225352
0.6971830985915493
0.5845070422535211
rogerstanimoto K-Nearest Neighbours
0.6408450704225352
************************************ ****************************
9
tanimoto_distance K-Nearest Neighbours
0.7253521126760564
0.7394366197183099
0.7183098591549296
0.6690140845070423
0.7253521126760564
0.6267605633802817
0.676056338028169
0.5845070422535211
rogerstanimoto K-Nearest Neighbours
0.6478873

In [92]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv"]
#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
###def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=200)")
selector = SelectKBest(score_func=f_classif, k=200)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
##x =(dict(zip(selector.scores_, word_list)))
##sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
##print(sorted_x)
##df = pd.DataFrame({'sorted_tokens' : sorted_x})
##df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
##print(len(fit.scores_))

#print(selector.scores_)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)
def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator


"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,15,2):
##        acc=[]
        print(k)
    
        """
        8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        print("tanimoto_distance K-Nearest Neighbours")
        print(metrics.accuracy_score(y_test, y_pred))
        
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "rogerstanimoto")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        print("rogerstanimoto K-Nearest Neighbours")
        print(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(round(mean(manhattanacc), 4))
print(round(mean(cosineacc), 4))
print(round(mean(euclideanacc), 4))
print(round(mean(braycurtisacc), 4))
print(round(mean(canberraacc), 4))
print(round(mean(jaccardacc), 4))
print(round(mean(yuleacc), 4))

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv 1063
/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv 1109
length of features used = 2172
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
selector = SelectKBest(score_func=f_classif, k=200)
3
tanimoto_distance K-Nearest Neighbours
0.823943661971831
0.7394366197183099
0.7605633802816901
0.7605633802816901
0.823943661971831
0.7253521126760564
0.6971830985915493
0.6830985915492958
rogerstanimoto K-Nearest Neighbours
0.7323943661971831
************************************ ****************************
5


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


tanimoto_distance K-Nearest Neighbours
0.823943661971831
0.7464788732394366
0.7323943661971831
0.7605633802816901
0.823943661971831
0.7323943661971831
0.7183098591549296
0.704225352112676
rogerstanimoto K-Nearest Neighbours
0.6830985915492958
************************************ ****************************
7
tanimoto_distance K-Nearest Neighbours
0.8028169014084507
0.7605633802816901
0.7183098591549296
0.7535211267605634
0.8028169014084507
0.7253521126760564
0.7183098591549296
0.6830985915492958
rogerstanimoto K-Nearest Neighbours
0.7605633802816901
************************************ ****************************
9
tanimoto_distance K-Nearest Neighbours
0.7816901408450704
0.7394366197183099
0.7253521126760564
0.7676056338028169
0.7816901408450704
0.7183098591549296
0.7112676056338029
0.6901408450704225
rogerstanimoto K-Nearest Neighbours
0.7676056338028169
************************************ ****************************
11
tanimoto_distance K-Nearest Neighbours
0.7323943661971831
0.

In [93]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]

txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015CHI.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015IG.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015GSS.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015OR.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015PMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015PMI.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015RF.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015RF.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015WLLR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015WLLR.2F.txt"]

print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
        print(txt_file, len(word_list))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
###def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=200)")
#selector = SelectKBest(score_func=f_classif, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
##x =(dict(zip(selector.scores_, word_list)))
##sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
##print(sorted_x)
##df = pd.DataFrame({'sorted_tokens' : sorted_x})
##df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
##print(len(fit.scores_))

#print(selector.scores_)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)
def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator


"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,15,2):
##        acc=[]
        print(k)
    
        """
        8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        print("tanimoto_distance K-Nearest Neighbours")
        print(metrics.accuracy_score(y_test, y_pred))
        
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "rogerstanimoto")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        print("rogerstanimoto K-Nearest Neighbours")
        print(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(round(mean(manhattanacc), 4))
print(round(mean(cosineacc), 4))
print(round(mean(euclideanacc), 4))
print(round(mean(braycurtisacc), 4))
print(round(mean(canberraacc), 4))
print(round(mean(jaccardacc), 4))
print(round(mean(yuleacc), 4))

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015CHI.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015CHI.txt 200
length of features used = 1309
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
selector = SelectKBest(score_func=f_classif, k=200)
3
tanimoto_distance K-Nearest Neighbours
0.7746478873239436
0.6267605633802817
0.7253521126760564
0.7183098591549296
0.7746478873239436
0.528169014084507
0.7183098591549296
0.5774647887323944
rogerstanimoto K-Nearest Neighbours
0.528169014084507
************************************ ****************************
5


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


tanimoto_distance K-Nearest Neighbours
0.7605633802816901
0.5633802816901409
0.704225352112676
0.6690140845070423
0.7605633802816901
0.5140845070422535
0.6690140845070423
0.5704225352112676
rogerstanimoto K-Nearest Neighbours
0.5211267605633803
************************************ ****************************
7
tanimoto_distance K-Nearest Neighbours
0.7605633802816901
0.528169014084507
0.7183098591549296
0.6408450704225352
0.7605633802816901
0.5140845070422535
0.6267605633802817
0.5352112676056338
rogerstanimoto K-Nearest Neighbours
0.528169014084507
************************************ ****************************
9
tanimoto_distance K-Nearest Neighbours
0.6901408450704225
0.5070422535211268
0.6549295774647887
0.6549295774647887
0.6901408450704225
0.5352112676056338
0.6338028169014085
0.528169014084507
rogerstanimoto K-Nearest Neighbours
0.5211267605633803
************************************ ****************************
11
tanimoto_distance K-Nearest Neighbours
0.6690140845070423
0.5

In [94]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]

#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015CHI.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015IG.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015GSS.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015OR.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015PMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015PMI.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015RF.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015RF.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015WLLR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015WLLR.2F.txt"]

print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
        print(txt_file, len(word_list))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
###def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=200)")
#selector = SelectKBest(score_func=f_classif, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
##x =(dict(zip(selector.scores_, word_list)))
##sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
##print(sorted_x)
##df = pd.DataFrame({'sorted_tokens' : sorted_x})
##df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
##print(len(fit.scores_))

#print(selector.scores_)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)
def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator


"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,15,2):
##        acc=[]
        print(k)
    
        """
        8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        print("tanimoto_distance K-Nearest Neighbours")
        print(metrics.accuracy_score(y_test, y_pred))
        
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "rogerstanimoto")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        print("rogerstanimoto K-Nearest Neighbours")
        print(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(round(mean(manhattanacc), 4))
print(round(mean(cosineacc), 4))
print(round(mean(euclideanacc), 4))
print(round(mean(braycurtisacc), 4))
print(round(mean(canberraacc), 4))
print(round(mean(jaccardacc), 4))
print(round(mean(yuleacc), 4))

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015IG.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015IG.txt 200
length of features used = 1309
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
selector = SelectKBest(score_func=f_classif, k=200)
3
tanimoto_distance K-Nearest Neighbours
0.7746478873239436
0.6267605633802817
0.7253521126760564
0.7183098591549296
0.7746478873239436
0.528169014084507
0.7183098591549296
0.5774647887323944
rogerstanimoto K-Nearest Neighbours
0.528169014084507
************************************ ****************************
5


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


tanimoto_distance K-Nearest Neighbours
0.7605633802816901
0.5633802816901409
0.704225352112676
0.6690140845070423
0.7605633802816901
0.5140845070422535
0.6690140845070423
0.5704225352112676
rogerstanimoto K-Nearest Neighbours
0.5211267605633803
************************************ ****************************
7
tanimoto_distance K-Nearest Neighbours
0.7605633802816901
0.528169014084507
0.7183098591549296
0.6408450704225352
0.7605633802816901
0.5140845070422535
0.6267605633802817
0.5352112676056338
rogerstanimoto K-Nearest Neighbours
0.528169014084507
************************************ ****************************
9
tanimoto_distance K-Nearest Neighbours
0.6901408450704225
0.5070422535211268
0.6549295774647887
0.6549295774647887
0.6901408450704225
0.5352112676056338
0.6338028169014085
0.528169014084507
rogerstanimoto K-Nearest Neighbours
0.5211267605633803
************************************ ****************************
11
tanimoto_distance K-Nearest Neighbours
0.6690140845070423
0.5

In [95]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]

#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015CHI.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015IG.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015GSS.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015OR.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015PMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015PMI.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015RF.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015RF.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015WLLR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015WLLR.2F.txt"]

print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
        print(txt_file, len(word_list))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
###def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=200)")
#selector = SelectKBest(score_func=f_classif, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
##x =(dict(zip(selector.scores_, word_list)))
##sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
##print(sorted_x)
##df = pd.DataFrame({'sorted_tokens' : sorted_x})
##df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
##print(len(fit.scores_))

#print(selector.scores_)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)
def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator


"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,15,2):
##        acc=[]
        print(k)
    
        """
        8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        print("tanimoto_distance K-Nearest Neighbours")
        print(metrics.accuracy_score(y_test, y_pred))
        
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "rogerstanimoto")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        print("rogerstanimoto K-Nearest Neighbours")
        print(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(round(mean(manhattanacc), 4))
print(round(mean(cosineacc), 4))
print(round(mean(euclideanacc), 4))
print(round(mean(braycurtisacc), 4))
print(round(mean(canberraacc), 4))
print(round(mean(jaccardacc), 4))
print(round(mean(yuleacc), 4))

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015GSS.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015GSS.txt 200
length of features used = 1309
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
selector = SelectKBest(score_func=f_classif, k=200)
3
tanimoto_distance K-Nearest Neighbours
0.7746478873239436
0.6197183098591549
0.7112676056338029
0.7183098591549296
0.7746478873239436
0.528169014084507
0.704225352112676
0.5845070422535211
rogerstanimoto K-Nearest Neighbours
0.528169014084507
************************************ ****************************
5


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


tanimoto_distance K-Nearest Neighbours
0.7535211267605634
0.5633802816901409
0.6901408450704225
0.6901408450704225
0.7535211267605634
0.5140845070422535
0.6830985915492958
0.5704225352112676
rogerstanimoto K-Nearest Neighbours
0.5211267605633803
************************************ ****************************
7
tanimoto_distance K-Nearest Neighbours
0.7464788732394366
0.5211267605633803
0.7112676056338029
0.6338028169014085
0.7464788732394366
0.5140845070422535
0.6338028169014085
0.5422535211267606
rogerstanimoto K-Nearest Neighbours
0.528169014084507
************************************ ****************************
9
tanimoto_distance K-Nearest Neighbours
0.704225352112676
0.5070422535211268
0.6549295774647887
0.647887323943662
0.704225352112676
0.5352112676056338
0.647887323943662
0.5352112676056338
rogerstanimoto K-Nearest Neighbours
0.5211267605633803
************************************ ****************************
11
tanimoto_distance K-Nearest Neighbours
0.6830985915492958
0.50

In [96]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]

#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015CHI.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015IG.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015GSS.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015OR.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015PMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015PMI.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015RF.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015RF.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015WLLR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015WLLR.2F.txt"]

print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list =  ([(r.split(":"))[1] for r in f])[0:100]
        print(txt_file, len(word_list))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
###def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=200)")
#selector = SelectKBest(score_func=f_classif, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
##x =(dict(zip(selector.scores_, word_list)))
##sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
##print(sorted_x)
##df = pd.DataFrame({'sorted_tokens' : sorted_x})
##df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
##print(len(fit.scores_))

#print(selector.scores_)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)
def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator


"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,15,2):
##        acc=[]
        print(k)
    
        """
        8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        print("tanimoto_distance K-Nearest Neighbours")
        print(metrics.accuracy_score(y_test, y_pred))
        
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "rogerstanimoto")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        print("rogerstanimoto K-Nearest Neighbours")
        print(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(round(mean(manhattanacc), 4))
print(round(mean(cosineacc), 4))
print(round(mean(euclideanacc), 4))
print(round(mean(braycurtisacc), 4))
print(round(mean(canberraacc), 4))
print(round(mean(jaccardacc), 4))
print(round(mean(yuleacc), 4))

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015OR.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015OR.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015OR.1M.txt 100
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015OR.2F.txt 100
length of features used = 1209
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
selector = SelectKBest(score_func=f_classif, k=200)
3
tanimoto_distance K-Nearest Neighbours
0.7746478873239436
0.6408450704225352
0.7112676056338029
0.7183098591549296
0.7746478873239436
0.528169014084507
0.7112676056338029
0.5704225352112676
rogerstanimoto K-Nearest Neighbours
0.528169014084507
************************************ ****************************
5


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


tanimoto_distance K-Nearest Neighbours
0.7676056338028169
0.5563380281690141
0.704225352112676
0.7112676056338029
0.7676056338028169
0.5140845070422535
0.6690140845070423
0.5704225352112676
rogerstanimoto K-Nearest Neighbours
0.5211267605633803
************************************ ****************************
7
tanimoto_distance K-Nearest Neighbours
0.6971830985915493
0.5211267605633803
0.6690140845070423
0.6549295774647887
0.6971830985915493
0.5140845070422535
0.6549295774647887
0.5352112676056338
rogerstanimoto K-Nearest Neighbours
0.5211267605633803
************************************ ****************************
9
tanimoto_distance K-Nearest Neighbours
0.6619718309859155
0.5140845070422535
0.6830985915492958
0.6267605633802817
0.6619718309859155
0.5422535211267606
0.6408450704225352
0.528169014084507
rogerstanimoto K-Nearest Neighbours
0.5211267605633803
************************************ ****************************
11
tanimoto_distance K-Nearest Neighbours
0.647887323943662
0.

In [97]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]

#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015CHI.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015IG.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015GSS.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015PMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015PMI.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015RF.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015RF.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015WLLR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015WLLR.2F.txt"]

print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list =  ([(r.split(":"))[1] for r in f])[0:100]
        print(txt_file, len(word_list))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
###def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=200)")
#selector = SelectKBest(score_func=f_classif, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
##x =(dict(zip(selector.scores_, word_list)))
##sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
##print(sorted_x)
##df = pd.DataFrame({'sorted_tokens' : sorted_x})
##df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
##print(len(fit.scores_))

#print(selector.scores_)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)
def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator


"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,15,2):
##        acc=[]
        print(k)
    
        """
        8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        print("tanimoto_distance K-Nearest Neighbours")
        print(metrics.accuracy_score(y_test, y_pred))
        
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "rogerstanimoto")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        print("rogerstanimoto K-Nearest Neighbours")
        print(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(round(mean(manhattanacc), 4))
print(round(mean(cosineacc), 4))
print(round(mean(euclideanacc), 4))
print(round(mean(braycurtisacc), 4))
print(round(mean(canberraacc), 4))
print(round(mean(jaccardacc), 4))
print(round(mean(yuleacc), 4))

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015PMI.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015PMI.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015PMI.1M.txt 100
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015PMI.2F.txt 100
length of features used = 1209
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
selector = SelectKBest(score_func=f_classif, k=200)
3
tanimoto_distance K-Nearest Neighbours
0.7746478873239436
0.6267605633802817
0.704225352112676
0.7253521126760564
0.7746478873239436
0.528169014084507
0.704225352112676
0.5845070422535211
rogerstanimoto K-Nearest Neighbours
0.528169014084507
************************************ ****************************
5


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


tanimoto_distance K-Nearest Neighbours
0.7464788732394366
0.5492957746478874
0.6971830985915493
0.7112676056338029
0.7464788732394366
0.5140845070422535
0.676056338028169
0.5704225352112676
rogerstanimoto K-Nearest Neighbours
0.5211267605633803
************************************ ****************************
7
tanimoto_distance K-Nearest Neighbours
0.6901408450704225
0.5140845070422535
0.6690140845070423
0.6338028169014085
0.6901408450704225
0.5140845070422535
0.6619718309859155
0.5352112676056338
rogerstanimoto K-Nearest Neighbours
0.5211267605633803
************************************ ****************************
9
tanimoto_distance K-Nearest Neighbours
0.6549295774647887
0.5070422535211268
0.6901408450704225
0.6267605633802817
0.6549295774647887
0.5352112676056338
0.6338028169014085
0.5211267605633803
rogerstanimoto K-Nearest Neighbours
0.5211267605633803
************************************ ****************************
11
tanimoto_distance K-Nearest Neighbours
0.647887323943662
0

In [98]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]

#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015CHI.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015IG.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015GSS.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015OR.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015PMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015PMI.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015RF.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015RF.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015WLLR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015WLLR.2F.txt"]

print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list =  ([(r.split(":"))[1] for r in f])[0:100]
        print(txt_file, len(word_list))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
###def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=200)")
#selector = SelectKBest(score_func=f_classif, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
##x =(dict(zip(selector.scores_, word_list)))
##sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
##print(sorted_x)
##df = pd.DataFrame({'sorted_tokens' : sorted_x})
##df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
##print(len(fit.scores_))

#print(selector.scores_)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)
def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator


"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,15,2):
##        acc=[]
        print(k)
    
        """
        8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        print("tanimoto_distance K-Nearest Neighbours")
        print(metrics.accuracy_score(y_test, y_pred))
        
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "rogerstanimoto")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        print("rogerstanimoto K-Nearest Neighbours")
        print(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(round(mean(manhattanacc), 4))
print(round(mean(cosineacc), 4))
print(round(mean(euclideanacc), 4))
print(round(mean(braycurtisacc), 4))
print(round(mean(canberraacc), 4))
print(round(mean(jaccardacc), 4))
print(round(mean(yuleacc), 4))

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015RF.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015RF.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015RF.1M.txt 100
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015RF.2F.txt 100
length of features used = 1209
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
selector = SelectKBest(score_func=f_classif, k=200)
3
tanimoto_distance K-Nearest Neighbours
0.7746478873239436
0.6267605633802817
0.704225352112676
0.7253521126760564
0.7746478873239436
0.528169014084507
0.704225352112676
0.5845070422535211
rogerstanimoto K-Nearest Neighbours
0.528169014084507
************************************ ****************************
5


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


tanimoto_distance K-Nearest Neighbours
0.7464788732394366
0.5492957746478874
0.6971830985915493
0.7112676056338029
0.7464788732394366
0.5140845070422535
0.676056338028169
0.5704225352112676
rogerstanimoto K-Nearest Neighbours
0.5211267605633803
************************************ ****************************
7
tanimoto_distance K-Nearest Neighbours
0.6901408450704225
0.5140845070422535
0.6690140845070423
0.6338028169014085
0.6901408450704225
0.5140845070422535
0.6619718309859155
0.5352112676056338
rogerstanimoto K-Nearest Neighbours
0.5211267605633803
************************************ ****************************
9
tanimoto_distance K-Nearest Neighbours
0.6549295774647887
0.5070422535211268
0.6901408450704225
0.6267605633802817
0.6549295774647887
0.5352112676056338
0.6338028169014085
0.5211267605633803
rogerstanimoto K-Nearest Neighbours
0.5211267605633803
************************************ ****************************
11
tanimoto_distance K-Nearest Neighbours
0.647887323943662
0

In [99]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]

#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015CHI.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015IG.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015GSS.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015OR.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015PMI.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015PMI.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015RF.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015RF.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015WLLR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015WLLR.2F.txt"]

print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
#        print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        word_list =  ([(r.split(":"))[1] for r in f])[0:100]
        print(txt_file, len(word_list))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2............................................................
"""
from sklearn.feature_selection import *
##
###def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=200)")
#selector = SelectKBest(score_func=f_classif, k=200)
#fit = selector.fit(X_train,y_train)
#train_selected_features = fit.transform(X_train)
##x =(dict(zip(selector.scores_, word_list)))
##sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
##print(sorted_x)
##df = pd.DataFrame({'sorted_tokens' : sorted_x})
##df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
##print(len(fit.scores_))

#print(selector.scores_)
#test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
#X_train =train_selected_features 
#X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)
def tanimoto_distance(x,y):
    """Calculates the tanimoto (normalised manhattan) distance between the vectors x and y
            Keyword arguments:
                x,y -- the vectors between which the distance is to be calculated
    """
    try:
        iter(x)
    except TypeError:
        logging.warning( 'Argument x is not iterable. None is returned')
        return None
    try:
        iter(y)
    except TypeError:
        logging.warning( 'Argument y is not iterable. None is returned')
        return None
    numerator = sum(abs(a - b) for a, b in zip(x, y))
    denominator = sum(max(a,b) for a,b in zip(x,y))
    return numerator/denominator


"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(3,15,2):
##        acc=[]
        print(k)
    
        """
        8.0. tanimoto_distance K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= tanimoto_distance)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        print("tanimoto_distance K-Nearest Neighbours")
        print(metrics.accuracy_score(y_test, y_pred))
        
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("manhattan K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        manhattanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. cosine K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "cosine")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("cosine K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        cosineacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        8. euclidean K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors = k, metric= "euclidean")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("euclidean K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        euclideanacc.append(metrics.accuracy_score(y_test, y_pred))
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "canberra")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("canberra K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        canberraacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "jaccard")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("jaccard K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        jaccardacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "yule")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("yule K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        yuleacc.append(metrics.accuracy_score(y_test, y_pred))
        
        knn = KNeighborsClassifier(n_neighbors = k, metric= "rogerstanimoto")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        print("rogerstanimoto K-Nearest Neighbours")
        print(metrics.accuracy_score(y_test, y_pred))
        print("************************************ ****************************")
print(round(mean(manhattanacc), 4))
print(round(mean(cosineacc), 4))
print(round(mean(euclideanacc), 4))
print(round(mean(braycurtisacc), 4))
print(round(mean(canberraacc), 4))
print(round(mean(jaccardacc), 4))
print(round(mean(yuleacc), 4))

txt_files of features used ['/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015WLLR.1M.txt', '/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015WLLR.2F.txt']
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015WLLR.1M.txt 100
/Users/catherine/Desktop/NLP/PAN Datasets/PAN_TermSelection/2015TermSelection/outputNewVersion2015WLLR.2F.txt 100
length of features used = 1209
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
selector = SelectKBest(score_func=f_classif, k=200)
3
tanimoto_distance K-Nearest Neighbours
0.7746478873239436
0.6267605633802817
0.704225352112676
0.7253521126760564
0.7746478873239436
0.528169014084507
0.704225352112676
0.5845070422535211
rogerstanimoto K-Nearest Neighbours
0.528169014084507
************************************ ****************************
5


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, DataConversionWarning)


tanimoto_distance K-Nearest Neighbours
0.7464788732394366
0.5492957746478874
0.6971830985915493
0.7112676056338029
0.7464788732394366
0.5140845070422535
0.676056338028169
0.5704225352112676
rogerstanimoto K-Nearest Neighbours
0.5211267605633803
************************************ ****************************
7
tanimoto_distance K-Nearest Neighbours
0.6901408450704225
0.5140845070422535
0.6690140845070423
0.6338028169014085
0.6901408450704225
0.5140845070422535
0.6619718309859155
0.5352112676056338
rogerstanimoto K-Nearest Neighbours
0.5211267605633803
************************************ ****************************
9
tanimoto_distance K-Nearest Neighbours
0.6549295774647887
0.5070422535211268
0.6901408450704225
0.6267605633802817
0.6549295774647887
0.5352112676056338
0.6338028169014085
0.5211267605633803
rogerstanimoto K-Nearest Neighbours
0.5211267605633803
************************************ ****************************
11
tanimoto_distance K-Nearest Neighbours
0.647887323943662
0

In [10]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv"]

#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2.f_classif.mutual_info_classif...........................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=chi2, k=200)")
selector = SelectKBest(score_func=chi2, k=200)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
#x =(dict(zip(selector.scores_, word_list)))
#sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
#print(sorted_x)
#df = pd.DataFrame({'sorted_tokens' : sorted_x})
#df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
#print(len(fit.scores_))

#print(selector.scores_)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(5,6,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
    
    
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        print([(i,j) for i, j in zip(y_pred, y_test)])
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
       
  
        print("************************************ ****************************")


txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv 1063
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv 1109
length of features used = 2172
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
selector = SelectKBest(score_func=chi2, k=200)
5
0.7535211267605634
[('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'

In [8]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv"]
#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2.f_classif.mutual_info_classif...........................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=f_classif, k=200)")
selector = SelectKBest(score_func=f_classif, k=200)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
#x =(dict(zip(selector.scores_, word_list)))
#sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
#print(sorted_x)
#df = pd.DataFrame({'sorted_tokens' : sorted_x})
#df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
#print(len(fit.scores_))

#print(selector.scores_)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(5,6,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
    
    
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        print([(i,j) for i, j in zip(y_pred, y_test)])
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
       
  
        print("************************************ ****************************")



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv 1063
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv 1109
length of features used = 2172
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
selector = SelectKBest(score_func=f_classif, k=200)
5
0.823943661971831
[('M', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('F', 'M'), ('M', 'M'), ('M',

In [9]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv"]

#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

"""
Perform feature selection using chi2.f_classif.mutual_info_classif...........................................................
"""
from sklearn.feature_selection import *
##
####def select_Bestfeatures(X, y):
print("selector = SelectKBest(score_func=mutual_info_classif, k=200)")
selector = SelectKBest(score_func=mutual_info_classif, k=200)
fit = selector.fit(X_train,y_train)
train_selected_features = fit.transform(X_train)
#x =(dict(zip(selector.scores_, word_list)))
#sorted_x = sorted(x.items(), key=operator.itemgetter(0), reverse=True)
#print(sorted_x)
#df = pd.DataFrame({'sorted_tokens' : sorted_x})
#df.to_csv('/Users/catherine/Desktop/NLP/PAN Datasets/PAN2015/2015txtfiles/2015_tweet_mutual_info_classif.csv', index=False, encoding='utf-8')
#print([for x in list(sorted_x)])
#print(len(fit.scores_))

#print(selector.scores_)
test_selected_features = fit.transform(X_test)



"""
x_train and x_test based on the new features using chi2............................................................
"""
##X_train = select_Bestfeatures(X_train, y_train)
##X_test = select_Bestfeatures(X_test, y_test)


"""
Perform feature selection using f_classif............................................................
"""
from sklearn.feature_selection import *
##def f_classif_select_Bestfeatures(X, y):
##selector=SelectKBest(score_func=f_classif,k=500)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = f_classif_select_Bestfeatures(X_train, y_train)
##X_test = f_classif_select_Bestfeatures(X_test, y_test)



"""
Perform feature selection using mutual_info_classif............................................................
"""
from sklearn.feature_selection import *
##def mutual_info_classif_select_Bestfeatures(X_tr, X_te, y):
##selector=SelectKBest(score_func=mutual_info_classif,k=100)
##fit = selector.fit(X_train,y_train)
##train_selected_features = fit.transform(X_train)
##test_selected_features = fit.transform(X_test)
##    return train_selected_features, test_selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
X_train =train_selected_features 
X_test = test_selected_features


"""
Perform feature selection using pca............................................................
"""

##from sklearn.decomposition import PCA
##def PCA_select_Bestfeatures(X):
##    # feature extraction
##    pca = PCA(n_components=200)
##    fit = pca.fit(X, y)
##    Train_selected_features = fit.transform(X)
##    return selected_features

"""
x_train and x_test based on the new features using f_classif............................................................
"""
##X_train = PCA_select_Bestfeatures(X_train)
##X_test = PCA_select_Bestfeatures(X_test)

"""
Below are a list of Classification Algorithms with different distance measres ..............................................................................
"""
"""
K-Nearest Neighbours (average aa over all the nearest neighbour selectiopn)..............................................................................
"""
manhattanacc=[]
cosineacc=[]
euclideanacc=[]
braycurtisacc=[]
canberraacc=[]
jaccardacc=[]
yuleacc=[]
from statistics import mean
for k in range(5,6,2):
##        acc=[]
        print(k)
        """
        8. manhattan K-Nearest Neighbours..............................................................................
        """
        from sklearn.neighbors import KNeighborsClassifier
    
    
        """
        9. scipy.spatial.distance K-Nearest Neighbours..............................................................................
        """
        from scipy.spatial.distance import *
        knn = KNeighborsClassifier(n_neighbors = k, metric= "braycurtis")
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        #print("braycurtis K-Nearest Neighbours",y_pred)
        print(metrics.accuracy_score(y_test, y_pred))
        print([(i,j) for i, j in zip(y_pred, y_test)])
        braycurtisacc.append(metrics.accuracy_score(y_test, y_pred))
        
       
  
        print("************************************ ****************************")




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv 1063
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv 1109
length of features used = 2172
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
selector = SelectKBest(score_func=mutual_info_classif, k=200)
5
0.5985915492957746
[('M', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('F', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('F', 'M'), ('F', 'M'), ('M', 'M'), ('F', 'M'), ('M', 'M'), ('M', 'M'), ('F', 'M'), ('F', 'M'), ('M', 

In [1]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv"]

#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)


print("************************************ ****************************")


k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=5************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

k=13

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=13************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("****************************************************************")


from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC



print("Naive****************************************************************")
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train, y_train)
y_pred = Naive.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("SVM****************************************************************")
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train, y_train)
y_pred = SVM.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("RandomForest_100****************************************************************")
clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

print("RandomForest_200****************************************************************")
clf=RandomForestClassifier(n_estimators=200)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))






/anaconda3/lib/python3.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv 1063
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv 1109
length of features used = 2172
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
************************************ ****************************
5
manhattan_k=5************************************ ****************************
manhattan K-Nearest Neighbours
0.7464788732394366
************************************ ****************************
13
manhattan_k=13************************************ ****************************
manhattan K-Nearest Neighbours
0

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


0.5915492957746479
RandomForest_100****************************************************************
0.7746478873239436
RandomForest_200****************************************************************
0.7535211267605634


In [2]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_vocubulary.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_ vocubulary_set.1.csv"]

#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)


print("************************************ ****************************")


k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=5************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

k=13

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=13************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("****************************************************************")


from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC



print("Naive****************************************************************")
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train, y_train)
y_pred = Naive.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("SVM****************************************************************")
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train, y_train)
y_pred = SVM.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("RandomForest_100****************************************************************")
clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

print("RandomForest_200****************************************************************")
clf=RandomForestClassifier(n_estimators=200)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_vocubulary.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_ vocubulary_set.1.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_vocubulary.1.csv 11167
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_ vocubulary_set.1.csv 11808
length of features used = 22975
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
************************************ ****************************
5
manhattan_k=5************************************ ****************************
manhattan K-Nearest Neighbours
0.795774647887324
************************************ ****************************
13
manhattan_k=13************************************ ****************************
manhattan K-Nearest 

In [7]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_vocubulary.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_ vocubulary_set.1.csv"]

#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)


print("************************************ ****************************")


k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=5************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
manhattan_k5voc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

k=13

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=13************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
manhattan_k13voc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))
print("****************************************************************")


from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier


print("MLPClassifier****************************************************************")
clf = MLPClassifier()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
MLPClassifiervoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("LogisticRegression****************************************************************")
clf = LogisticRegression()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
LogisticRegressionvoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("SGDClassifier****************************************************************")
clf = SGDClassifier()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
SGDClassifiervoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("DecisionTreeClassifier****************************************************************")
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()
# Train Decision Tree Classifer
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
DecisionTreeClassifiervoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("Naive****************************************************************")
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train, y_train)
y_pred = Naive.predict(X_test)
Naivevoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("SVM****************************************************************")
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train, y_train)
y_pred = SVM.predict(X_test)
SVMvoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("RandomForest_100****************************************************************")
clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
RandomForest_100voc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))

print("RandomForest_200****************************************************************")
clf=RandomForestClassifier(n_estimators=200)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
RandomForest_200voc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))

from xgboost.sklearn import XGBClassifier 
print("XGBClassifier****************************************************************")
clf=XGBClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
XGBClassifiervoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))



print("BaggingClassifier****************************************************************")
from sklearn.ensemble.bagging import BaggingClassifier
clf=BaggingClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
BaggingClassifiervoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("AdaBoostClassifier****************************************************************")
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
clf=AdaBoostClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
AdaBoostClassifiervoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))








txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_vocubulary.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_ vocubulary_set.1.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_vocubulary.1.csv 11167
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_ vocubulary_set.1.csv 11808
length of features used = 22975
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
************************************ ****************************
5
manhattan_k=5************************************ ****************************
manhattan K-Nearest Neighbours
0.795774647887324
************************************ ****************************
13
manhattan_k=13************************************ ****************************
manhattan K-Nearest 

/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.8098591549295775
LogisticRegression****************************************************************
0.5774647887323944
SGDClassifier****************************************************************
0.5985915492957746
DecisionTreeClassifier****************************************************************
0.6197183098591549
Naive****************************************************************
0.6408450704225352
SVM****************************************************************
0.5915492957746479
RandomForest_100****************************************************************
0.7535211267605634
RandomForest_200****************************************************************
0.7746478873239436
XGBClassifier****************************************************************


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.7605633802816901
BaggingClassifier****************************************************************
0.676056338028169
AdaBoostClassifier****************************************************************
0.7464788732394366


In [11]:
print("The sign test...****************************************************************")
print("manhattan_k=5************************************ ****************************")
x= MLPClassifiervoc 
y= manhattan_k5voc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)
        
print("manhattan_k=13************************************ ****************************")
x= MLPClassifiervoc 
y= manhattan_k13voc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)

print("MLPClassifier****************************************************************")
x= MLPClassifiervoc 
y= MLPClassifiervoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("LogisticRegression****************************************************************")
x= MLPClassifiervoc 
y= LogisticRegressionvoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("SGDClassifier****************************************************************")
x= MLPClassifiervoc 
y= SGDClassifiervoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("DecisionTreeClassifier****************************************************************")
x= MLPClassifiervoc 
y= DecisionTreeClassifiervoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("Naive****************************************************************")
x= MLPClassifiervoc 
y= Naivevoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("SVM****************************************************************")
x= MLPClassifiervoc 
y= SVMvoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("RandomForest_100****************************************************************")
x= MLPClassifiervoc 
y= RandomForest_100voc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("RandomForest_200****************************************************************")
x= MLPClassifiervoc 
y= RandomForest_200voc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("XGBClassifier****************************************************************")
x= MLPClassifiervoc 
y= XGBClassifiervoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("BaggingClassifier****************************************************************")
x= MLPClassifiervoc 
y= BaggingClassifiervoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("AdaBoostClassifier****************************************************************")
x= MLPClassifiervoc
y= AdaBoostClassifiervoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)










The sign test...****************************************************************
manhattan_k=5************************************ ****************************
positive= 21 negative = 19
manhattan_k=13************************************ ****************************
positive= 32 negative = 6
MLPClassifier****************************************************************
positive= 0 negative = 0
LogisticRegression****************************************************************
positive= 40 negative = 7
SGDClassifier****************************************************************
positive= 39 negative = 9
DecisionTreeClassifier****************************************************************
positive= 39 negative = 12
Naive****************************************************************
positive= 35 negative = 11
SVM****************************************************************
positive= 39 negative = 8
RandomForest_100****************************************************************
posit

In [6]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv"]

#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)


print("************************************ ****************************")


k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=5************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
manhattan_k5fs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

k=13

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=13************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
manhattan_k13fs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))
print("****************************************************************")


from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier


print("MLPClassifier****************************************************************")
clf = MLPClassifier()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
MLPClassifierfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("LogisticRegression****************************************************************")
clf = LogisticRegression()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
LogisticRegressionfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("SGDClassifier****************************************************************")
clf = SGDClassifier()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
SGDClassifierfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("DecisionTreeClassifier****************************************************************")
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()
# Train Decision Tree Classifer
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
DecisionTreeClassifierfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("Naive****************************************************************")
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train, y_train)
y_pred = Naive.predict(X_test)
Naivefs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("SVM****************************************************************")
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train, y_train)
y_pred = SVM.predict(X_test)
SVMfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("RandomForest_100****************************************************************")
clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
RandomForest_100fs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))

print("RandomForest_200****************************************************************")
clf=RandomForestClassifier(n_estimators=200)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
RandomForest_200fs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))

from xgboost.sklearn import XGBClassifier 
print("XGBClassifier****************************************************************")
clf=XGBClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
XGBClassifierfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))



print("BaggingClassifier****************************************************************")
from sklearn.ensemble.bagging import BaggingClassifier
clf=BaggingClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
BaggingClassifierfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("AdaBoostClassifier****************************************************************")
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
clf=AdaBoostClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
AdaBoostClassifierfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))




txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv 1063
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv 1109
length of features used = 2172
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
************************************ ****************************
5
manhattan_k=5************************************ ****************************
manhattan K-Nearest Neighbours
0.7464788732394366
************************************ ****************************
13
manhattan_k=13************************************ ****************************
manhattan K-Nearest Neighbours
0

/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.7394366197183099
LogisticRegression****************************************************************
0.5704225352112676
SGDClassifier****************************************************************
0.5
DecisionTreeClassifier****************************************************************
0.6126760563380281
Naive****************************************************************
0.6338028169014085
SVM****************************************************************
0.5915492957746479
RandomForest_100****************************************************************


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.7816901408450704
RandomForest_200****************************************************************
0.7887323943661971
XGBClassifier****************************************************************


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.7676056338028169
BaggingClassifier****************************************************************
0.6267605633802817
AdaBoostClassifier****************************************************************
0.7464788732394366


In [10]:
print("The sign test...****************************************************************")
print("manhattan_k=5************************************ ****************************")
x= RandomForest_200fs 
y= manhattan_k5fs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)
        
print("manhattan_k=13************************************ ****************************")
x= RandomForest_200fs 
y= manhattan_k13fs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)

print("MLPClassifier****************************************************************")
x= RandomForest_200fs 
y= MLPClassifierfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("LogisticRegression****************************************************************")
x= RandomForest_200fs
y= LogisticRegressionfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("SGDClassifier****************************************************************")
x= RandomForest_200fs 
y= SGDClassifierfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("DecisionTreeClassifier****************************************************************")
x= RandomForest_200fs 
y= DecisionTreeClassifierfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("Naive****************************************************************")
x= RandomForest_200fs 
y= Naivefs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("SVM****************************************************************")
x= RandomForest_200fs 
y= SVMfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("RandomForest_100****************************************************************")
x= RandomForest_200fs 
y= RandomForest_100fs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("RandomForest_200****************************************************************")
x= RandomForest_200fs 
y= RandomForest_200fs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("XGBClassifier****************************************************************")
x= RandomForest_200fs 
y= XGBClassifierfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("BaggingClassifier****************************************************************")
x= RandomForest_200fs 
y= BaggingClassifierfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("AdaBoostClassifier****************************************************************")
x= RandomForest_200fs
y= AdaBoostClassifierfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




The sign test...****************************************************************
manhattan_k=5************************************ ****************************
positive= 24 negative = 18
manhattan_k=13************************************ ****************************
positive= 26 negative = 15
MLPClassifier****************************************************************
positive= 16 negative = 9
LogisticRegression****************************************************************
positive= 39 negative = 8
SGDClassifier****************************************************************
positive= 54 negative = 13
DecisionTreeClassifier****************************************************************
positive= 37 negative = 12
Naive****************************************************************
positive= 33 negative = 11
SVM****************************************************************
positive= 36 negative = 8
RandomForest_100****************************************************************
po

In [5]:
#..................................................................................

data_tags = ["type","gender","tweet"]
train_data_list = []
i=0
for f in y_train:
   
    train_data_list.append((f, y_train[i],all_training_text[i]))
    i += 1
# We have training data available as dictionary filename, category, data
train_data = pd.DataFrame.from_records(train_data_list, columns=data_tags)




#..................................................................................

test_data_list = []
ii=0
for f in y_test:
   
    test_data_list.append((f, y_test[ii], all_test_text[ii]))
    ii += 1
# We have training data available as dictionary filename, category, data
test_data = pd.DataFrame.from_records(test_data_list, columns=data_tags)

#..................................................................................


import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path

Using TensorFlow backend.
/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


In [6]:

train_posts = train_data['tweet']
train_tags = train_data['type']

test_posts = test_data['tweet']
test_tags = test_data['type']





In [7]:
# :Tokenize and Prepare Vocabulary
num_labels = 2
#vocab_size = 15000#len(word_list)#most common number of words will be then kept for use in the vector
vocab_size = 1000#by changing the vocubulary size... acc count_type =  0.932258064516129#acc count_gender = 0.8298387096774194
#vocab_size = 100#acc count_type =0.9274193548387096 acc count_gender = 0.792741935483871
batch_size = 100
 
# define Tokenizer with Vocab Size
tokenizer = Tokenizer(num_words=vocab_size)#IF I USE OTHER TOKENIZER...
tokenizer.fit_on_texts(train_posts)
#print(tokenizer.word_index)
#print((tokenizer.word_counts))#provides a dictionary of the words and the count......................
#sorted_x = sorted((tokenizer.word_counts).items(), key=operator.itemgetter(1),reverse=True)
#print(sorted_x)
#print((tokenizer.document_count))#number of documents..............
#print((tokenizer.word_docs))#provides a dictionary of the words and the number of documents they appear in......................
print('Found %d unique words.' % len(tokenizer.word_index))#shows total vocubulary of the text dataset
x_train = tokenizer.texts_to_matrix(train_posts, mode='tfidf')#WHAT HAPPENS WHEN I GIVE IT THE HAND CRAFTED TOKENS??
x_test = tokenizer.texts_to_matrix(test_posts, mode='tfidf')

##--------what about applying knn at this point-------------------------------
##--------what about changing the vocubulary size-------------------------------works well with reduced sized
##----------One popular method for hyperparameter optimization is grid search.-----------
##-----determine the best set of parameters with the highest accuracy..........this is for the keras model------


    
    
encoder = LabelBinarizer()


encoder.fit(train_tags)

y_train = encoder.transform(train_tags)#same as the y train generated with my model
y_train = np.hstack((y_train, 1 - y_train))#used for two label cases........

y_test = encoder.transform(test_tags)
y_test = np.hstack((y_test, 1 - y_test))

"""
fit_on_texts Updates internal vocabulary based on a list of texts. This method creates the vocabulary index based on word frequency. So if you give it something like, "The cat sat on the mat." It will create a dictionary s.t. word_index["the"] = 0; word_index["cat"] = 1 it is word -> index dictionary so every word gets a unique integer value. So lower integer means more frequent word (often the first few are punctuation because they appear a lot).
texts_to_sequences Transforms each text in texts to a sequence of integers. So it basically takes each word in the text and replaces it with its corresponding integer value from the word_index dictionary. Nothing more, nothing less, certainly no magic involved.
CLEAN TEXT TO THE DESIRED LEVEL AND USE KERAS INBUILF TFIDF TO GENERATE A MATRIX....
"""

Found 18373 unique words.


'\nfit_on_texts Updates internal vocabulary based on a list of texts. This method creates the vocabulary index based on word frequency. So if you give it something like, "The cat sat on the mat." It will create a dictionary s.t. word_index["the"] = 0; word_index["cat"] = 1 it is word -> index dictionary so every word gets a unique integer value. So lower integer means more frequent word (often the first few are punctuation because they appear a lot).\ntexts_to_sequences Transforms each text in texts to a sequence of integers. So it basically takes each word in the text and replaces it with its corresponding integer value from the word_index dictionary. Nothing more, nothing less, certainly no magic involved.\nCLEAN TEXT TO THE DESIRED LEVEL AND USE KERAS INBUILF TFIDF TO GENERATE A MATRIX....\n'

In [8]:
from keras.models import *
from keras.layers import *

model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))#512 neurons in the first hidden layer
model.add(Activation('relu'))
#model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
#model.add(Dropout(0.3))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()
 
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=30,
                    verbose=1,
                    validation_split=0.1)


#epochs... number of epochs should not be too large to cause overfitting.......

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               512512    
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_2 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 1026      
_________________________________________________________________
activation_3 (Activation)    (None, 2)                 0         
Total params: 776,194
Trainable params: 776,194
Non-trainable params: 0
_________________________________________________________________
Trai

In [9]:
#Evaluate model.............................
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
 
print('Test accuracy:', score[1])

142/142 [==============================] - 0s 35us/step
Test accuracy: 0.7816901349685561


In [3]:
#With Hyper Parameters Tuning
#DesicionTree

import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv"]

#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)


print("************************************ ****************************........................")



from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier




print("DecisionTreeClassifier****************************************************************")
# Create Decision Tree classifer object
#clf = DecisionTreeClassifier()
# Train Decision Tree Classifer
#clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
#y_pred = clf.predict(X_test)
#print(metrics.accuracy_score(y_test, y_pred))

#importing modules
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
#making the instance
model= DecisionTreeClassifier(random_state=1234)
#Hyper Parameters Set
params = {'max_features': ['auto', 'sqrt', 'log2'],
          'min_samples_split': [2,3,4,5,6,7,8,9,10,11,12,13,14,15], 
          'min_samples_leaf':[1,2,3,4,5,6,7,8,9,10,11],
          'random_state':[123]}
#Making models with hyper parameters sets
model1 = GridSearchCV(model, param_grid=params, n_jobs=-1)
#Learning
model1.fit(X_train, y_train)
#The best hyper parameters set
print("Best Hyper Parameters:",model1.best_params_)
#Prediction
prediction=model1.predict(X_test)
#importing the metrics module
from sklearn import metrics
#evaluation(Accuracy)
print("Accuracy:",metrics.accuracy_score(prediction,y_test))
#evaluation(Confusion Metrix)
print("Confusion Metrix:\n",metrics.confusion_matrix(prediction,y_test))

print("Randomforest****************************************************************")
#With Hyper Parameters Tuning
#Randomforest
#importing modules
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
#making the instance
model=RandomForestClassifier()
#hyper parameters set
params = {'criterion':['gini','entropy'],
          'n_estimators':[10,15,20,25,30],
          'min_samples_leaf':[1,2,3],
          'min_samples_split':[3,4,5,6,7], 
          'random_state':[123],
          'n_jobs':[-1]}
#Making models with hyper parameters sets
model1 = GridSearchCV(model, param_grid=params, n_jobs=-1)
#learning
model1.fit(X_train, y_train)
#The best hyper parameters set
print("Best Hyper Parameters:\n",model1.best_params_)
#Prediction
prediction=model1.predict(X_test)
#importing the metrics module
from sklearn import metrics
#evaluation(Accuracy)
print("Accuracy:",metrics.accuracy_score(prediction,y_test))
#evaluation(Confusion Metrix)
print("Confusion Metrix:\n",metrics.confusion_matrix(prediction,y_test))


print("RSVM****************************************************************")
#With Hyper Parameters Tuning
#SVM
#importing modules
from sklearn.model_selection import GridSearchCV
from sklearn import svm
#making the instance
model=svm.SVC()
#Hyper Parameters Set
params = {'C': [6,7,8,9,10,11,12], 
          'kernel': ['linear','rbf']}
#Making models with hyper parameters sets
model1 = GridSearchCV(model, param_grid=params, n_jobs=-1)
#Learning
model1.fit(X_train, y_train)
#The best hyper parameters set
print("Best Hyper Parameters:\n",model1.best_params_)
#Prediction
prediction=model1.predict(X_test)
#importing the metrics module
from sklearn import metrics
#evaluation(Accuracy)
print("Accuracy:",metrics.accuracy_score(prediction,y_test))
#evaluation(Confusion Metrix)
print("Confusion Metrix:\n",metrics.confusion_matrix(prediction,y_test))



print("kNearestNeighbors****************************************************************")
#With Hyper Parameters Tuning
#kNearestNeighbors
#importing modules
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
#making the instance
model = KNeighborsClassifier(n_jobs=-1)
#Hyper Parameters Set
params = {'n_neighbors':[5,6,7,8,9,10,13,15],
          'leaf_size':[1,2,3,5],
          'weights':['uniform', 'distance'],
          'algorithm':['auto', 'ball_tree','kd_tree','brute'],
          'n_jobs':[-1]}
#Making models with hyper parameters sets
model1 = GridSearchCV(model, param_grid=params, n_jobs=1)
#Learning
model1.fit(X_train, y_train)
#The best hyper parameters set
print("Best Hyper Parameters:\n",model1.best_params_)
#Prediction
prediction=model1.predict(X_test)
#importing the metrics module
from sklearn import metrics
#evaluation(Accuracy)
print("Accuracy:",metrics.accuracy_score(prediction,y_test))
#evaluation(Confusion Metrix)
print("Confusion Metrix:\n",metrics.confusion_matrix(prediction,y_test))



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv 1063
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv 1109
length of features used = 2172
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
************************************ ****************************........................
DecisionTreeClassifier****************************************************************
Best Hyper Parameters: {'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 11, 'random_state': 123}
Accuracy: 0.47183098591549294
Confusion Metrix:
 [[41 45]
 [30 26]]
Randomforest****

In [12]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv"]

#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
#X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
#print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
#X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
#print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)


print("************************************ ****************************")


#..................................................................................

data_tags = ["type","gender","tweet"]
train_data_list = []
i=0
for f in y_train:
   
    train_data_list.append((f, y_train[i],all_training_text[i]))
    i += 1
# We have training data available as dictionary filename, category, data
train_data = pd.DataFrame.from_records(train_data_list, columns=data_tags)




#..................................................................................

test_data_list = []
ii=0
for f in y_test:
   
    test_data_list.append((f, y_test[ii], all_test_text[ii]))
    ii += 1
# We have training data available as dictionary filename, category, data
test_data = pd.DataFrame.from_records(test_data_list, columns=data_tags)

#..................................................................................


import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path

# :Tokenize and Prepare Vocabulary

train_posts = train_data['tweet']
train_tags = train_data['type']

test_posts = test_data['tweet']
test_tags = test_data['type']


num_labels = 2
vocab_size = 1000#len(word_list)#most common number of words will be then kept for use in the vector
#vocab_size = 1000#by changing the vocubulary size... acc count_type =  0.932258064516129#acc count_gender = 0.8298387096774194
#vocab_size = 100#acc count_type =0.9274193548387096 acc count_gender = 0.792741935483871
batch_size = 100
 
# define Tokenizer with Vocab Size
tokenizer = Tokenizer(num_words=vocab_size)#IF I USE OTHER TOKENIZER...
tokenizer.fit_on_texts(train_posts)
#print(tokenizer.word_index)
#print((tokenizer.word_counts))#provides a dictionary of the words and the count......................
#sorted_x = sorted((tokenizer.word_counts).items(), key=operator.itemgetter(1),reverse=True)
#print(sorted_x)
#print((tokenizer.document_count))#number of documents..............
#print((tokenizer.word_docs))#provides a dictionary of the words and the number of documents they appear in......................
print('Found %d unique words.' % len(tokenizer.word_index))#shows total vocubulary of the text dataset
x_train = tokenizer.texts_to_matrix(train_posts, mode='tfidf')#WHAT HAPPENS WHEN I GIVE IT THE HAND CRAFTED TOKENS??
x_test = tokenizer.texts_to_matrix(test_posts, mode='tfidf')

##--------what about applying knn at this point-------------------------------
##--------what about changing the vocubulary size-------------------------------works well with reduced sized
##----------One popular method for hyperparameter optimization is grid search.-----------
##-----determine the best set of parameters with the highest accuracy..........this is for the keras model------


      
encoder = LabelBinarizer()


encoder.fit(train_tags)

y_train = encoder.transform(train_tags)#same as the y train generated with my model
y_train = np.hstack((y_train, 1 - y_train))#used for two label cases........

y_test = encoder.transform(test_tags)
y_test = np.hstack((y_test, 1 - y_test))


"""
fit_on_texts Updates internal vocabulary based on a list of texts. This method creates the vocabulary index based on word frequency. So if you give it something like, "The cat sat on the mat." It will create a dictionary s.t. word_index["the"] = 0; word_index["cat"] = 1 it is word -> index dictionary so every word gets a unique integer value. So lower integer means more frequent word (often the first few are punctuation because they appear a lot).
texts_to_sequences Transforms each text in texts to a sequence of integers. So it basically takes each word in the text and replaces it with its corresponding integer value from the word_index dictionary. Nothing more, nothing less, certainly no magic involved.
CLEAN TEXT TO THE DESIRED LEVEL AND USE KERAS INBUILF TFIDF TO GENERATE A MATRIX....
"""
from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
#8. manhattan K-Nearest Neighbours..............................................................................


k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=5************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(x_train, train_tags)
y_pred = knn.predict(x_test)
manhattan_k5tfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(test_tags, y_pred))
print("************************************ ****************************")

k=13

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=13************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(x_train, train_tags)
y_pred = knn.predict(x_test)
manhattan_k13tfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(test_tags, y_pred))
print("****************************************************************")


from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier


print("MLPClassifier****************************************************************")
clf = MLPClassifier()
clf = clf.fit(x_train, train_tags)
#Predict the response for test dataset
y_pred = clf.predict(x_test)
MLPClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("LogisticRegression****************************************************************")
clf = LogisticRegression()
clf = clf.fit(x_train, train_tags)
#Predict the response for test dataset
y_pred = clf.predict(x_test)
LogisticRegressiontfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("SGDClassifier****************************************************************")
clf = SGDClassifier()
clf = clf.fit(x_train, train_tags)
#Predict the response for test dataset
y_pred = clf.predict(x_test)
SGDClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("DecisionTreeClassifier****************************************************************")
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()
# Train Decision Tree Classifer
clf = clf.fit(x_train, train_tags)
#Predict the response for test dataset
y_pred = clf.predict(x_test)
DecisionTreeClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("Naive****************************************************************")
Naive = naive_bayes.MultinomialNB()
Naive.fit(x_train, train_tags)
y_pred = Naive.predict(x_test)
Naivetfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("SVM****************************************************************")
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(x_train, train_tags)
y_pred = SVM.predict(x_test)
SVMtfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("RandomForest_100****************************************************************")
clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train,train_tags)
y_pred=clf.predict(x_test)
RandomForest_100tfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))

print("RandomForest_200****************************************************************")
clf=RandomForestClassifier(n_estimators=200)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train,train_tags)
y_pred=clf.predict(x_test)
RandomForest_200tfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))

from xgboost.sklearn import XGBClassifier 
print("XGBClassifier****************************************************************")
clf=XGBClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train, train_tags)
y_pred=clf.predict(x_test)
XGBClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))



print("BaggingClassifier****************************************************************")
from sklearn.ensemble.bagging import BaggingClassifier
clf=BaggingClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train, train_tags)
y_pred=clf.predict(x_test)
BaggingClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("AdaBoostClassifier****************************************************************")
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
clf=AdaBoostClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train, train_tags)
y_pred=clf.predict(x_test)
AdaBoostClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))





from keras.models import *
from keras.layers import *

model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))#512 neurons in the first hidden layer
model.add(Activation('relu'))
#model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
#model.add(Dropout(0.3))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()
 
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=30,
                    verbose=1,
                    validation_split=0.1)#.... check out the problem this region causes



#Evaluate model.............................
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
 
print('Test accuracy:', score[1])





txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv 1063
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv 1109
length of features used = 2172
len(all_training_text) 152
len(all_test_text) 142
************************************ ****************************
Found 18373 unique words.
5
manhattan_k=5************************************ ****************************
manhattan K-Nearest Neighbours
0.704225352112676
************************************ ****************************
13
manhattan_k=13************************************ ****************************
manhattan K-Nearest Neighbours
0.6619718309859155
**************************

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.823943661971831
RandomForest_200****************************************************************
0.8098591549295775
XGBClassifier****************************************************************


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.7464788732394366
BaggingClassifier****************************************************************
0.7746478873239436
AdaBoostClassifier****************************************************************
0.7183098591549296
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               512512    
_________________________________________________________________
activation_4 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_5 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 1026      
____________________________________________________

In [13]:
print("The sign test...****************************************************************")
print("manhattan_k=5************************************ ****************************")
x= SGDClassifiertfidf 
y= manhattan_k5tfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)
        
print("manhattan_k=13************************************ ****************************")
x= SGDClassifiertfidf 
y= manhattan_k13tfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)

print("MLPClassifier****************************************************************")
x= SGDClassifiertfidf 
y= MLPClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("LogisticRegression****************************************************************")
x= SGDClassifiertfidf 
y= LogisticRegressiontfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("SGDClassifier****************************************************************")
x= SGDClassifiertfidf 
y= SGDClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("DecisionTreeClassifier****************************************************************")
x= SGDClassifiertfidf 
y= DecisionTreeClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("Naive****************************************************************")
x= SGDClassifiertfidf 
y= Naivetfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("SVM****************************************************************")
x= SGDClassifiertfidf 
y= SVMtfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("RandomForest_100****************************************************************")
x= SGDClassifiertfidf 
y= RandomForest_100tfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("RandomForest_200****************************************************************")
x= SGDClassifiertfidf
y= RandomForest_200tfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("XGBClassifier****************************************************************")
x= SGDClassifiertfidf 
y= XGBClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("BaggingClassifier****************************************************************")
x= SGDClassifiertfidf 
y= BaggingClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("AdaBoostClassifier****************************************************************")
x= SGDClassifiertfidf
y= AdaBoostClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)










The sign test...****************************************************************
manhattan_k=5************************************ ****************************
positive= 30 negative = 6
manhattan_k=13************************************ ****************************
positive= 38 negative = 8
MLPClassifier****************************************************************
positive= 14 negative = 4
LogisticRegression****************************************************************
positive= 6 negative = 4
SGDClassifier****************************************************************
positive= 0 negative = 0
DecisionTreeClassifier****************************************************************
positive= 39 negative = 9
Naive****************************************************************
positive= 33 negative = 5
SVM****************************************************************
positive= 8 negative = 4
RandomForest_100****************************************************************
positive= 

In [ ]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv"]

#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)


print("************************************ ****************************")


#..................................................................................

data_tags = ["type","gender","tweet"]
train_data_list = []
i=0
for f in y_train:
   
    train_data_list.append((f, y_train[i],all_training_text[i]))
    i += 1
# We have training data available as dictionary filename, category, data
train_data = pd.DataFrame.from_records(train_data_list, columns=data_tags)




#..................................................................................

test_data_list = []
ii=0
for f in y_test:
   
    test_data_list.append((f, y_test[ii], all_test_text[ii]))
    ii += 1
# We have training data available as dictionary filename, category, data
test_data = pd.DataFrame.from_records(test_data_list, columns=data_tags)

#..................................................................................


import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path

# :Tokenize and Prepare Vocabulary

train_posts = train_data['tweet']
train_tags = train_data['type']

test_posts = test_data['tweet']
test_tags = test_data['type']


num_labels = 2
vocab_size = 1000#len(word_list)#most common number of words will be then kept for use in the vector
#vocab_size = 1000#by changing the vocubulary size... acc count_type =  0.932258064516129#acc count_gender = 0.8298387096774194
#vocab_size = 100#acc count_type =0.9274193548387096 acc count_gender = 0.792741935483871
batch_size = 100
 
# define Tokenizer with Vocab Size
tokenizer = Tokenizer(num_words=vocab_size)#IF I USE OTHER TOKENIZER...
tokenizer.fit_on_texts(train_posts)
#print(tokenizer.word_index)
#print((tokenizer.word_counts))#provides a dictionary of the words and the count......................
#sorted_x = sorted((tokenizer.word_counts).items(), key=operator.itemgetter(1),reverse=True)
#print(sorted_x)
#print((tokenizer.document_count))#number of documents..............
#print((tokenizer.word_docs))#provides a dictionary of the words and the number of documents they appear in......................
print('Found %d unique words.' % len(tokenizer.word_index))#shows total vocubulary of the text dataset
x_train = tokenizer.texts_to_matrix(train_posts, mode='tfidf')#WHAT HAPPENS WHEN I GIVE IT THE HAND CRAFTED TOKENS??
x_test = tokenizer.texts_to_matrix(test_posts, mode='tfidf')

##--------what about applying knn at this point-------------------------------
##--------what about changing the vocubulary size-------------------------------works well with reduced sized
##----------One popular method for hyperparameter optimization is grid search.-----------
##-----determine the best set of parameters with the highest accuracy..........this is for the keras model------


      
encoder = LabelBinarizer()


encoder.fit(train_tags)

y_train = encoder.transform(train_tags)#same as the y train generated with my model
y_train = np.hstack((y_train, 1 - y_train))#used for two label cases........

y_test = encoder.transform(test_tags)
y_test = np.hstack((y_test, 1 - y_test))


"""
fit_on_texts Updates internal vocabulary based on a list of texts. This method creates the vocabulary index based on word frequency. So if you give it something like, "The cat sat on the mat." It will create a dictionary s.t. word_index["the"] = 0; word_index["cat"] = 1 it is word -> index dictionary so every word gets a unique integer value. So lower integer means more frequent word (often the first few are punctuation because they appear a lot).
texts_to_sequences Transforms each text in texts to a sequence of integers. So it basically takes each word in the text and replaces it with its corresponding integer value from the word_index dictionary. Nothing more, nothing less, certainly no magic involved.
CLEAN TEXT TO THE DESIRED LEVEL AND USE KERAS INBUILF TFIDF TO GENERATE A MATRIX....
"""
from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
#8. manhattan K-Nearest Neighbours..............................................................................


k=5
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(x_train, train_tags)
y_pred = knn.predict(x_test)
print("k=5 manhattan K-Nearest Neighbours")
print("accuracy = ",metrics.accuracy_score(test_tags, y_pred))
print("****************************************************************")



k=13

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(x_train, train_tags)
y_pred = knn.predict(x_test)
print("k=13 manhattan K-Nearest Neighbours")
print("accuracy = ",metrics.accuracy_score(test_tags, y_pred))
print("****************************************************************")



print("MLPClassifier****************************************************************")
clf = MLPClassifier()
clf = clf.fit(x_train, train_tags)
#Predict the response for test dataset
y_pred = clf.predict(x_test)
print(metrics.accuracy_score(test_tags, y_pred))


print("LogisticRegression****************************************************************")
clf = LogisticRegression()
clf = clf.fit(x_train, train_tags)
#Predict the response for test dataset
y_pred = clf.predict(x_test)
print(metrics.accuracy_score(test_tags, y_pred))


print("SGDClassifier****************************************************************")
clf = SGDClassifier()
clf = clf.fit(x_train, train_tags)
#Predict the response for test dataset
y_pred = clf.predict(x_test)
print(metrics.accuracy_score(test_tags, y_pred))


print("DecisionTreeClassifier****************************************************************")
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()
# Train Decision Tree Classifer
clf = clf.fit(x_train, train_tags)
#Predict the response for test dataset
y_pred = clf.predict(x_test)
print(metrics.accuracy_score(test_tags, y_pred))


print("Naive****************************************************************")
Naive = naive_bayes.MultinomialNB()
Naive.fit(x_train, train_tags)
y_pred = Naive.predict(x_test)
print(metrics.accuracy_score(test_tags, y_pred))


print("SVM****************************************************************")
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(x_train, train_tags)
y_pred = SVM.predict(x_test)
print(metrics.accuracy_score(test_tags, y_pred))


print("RandomForest_100****************************************************************")
clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train, train_tags)
y_pred=clf.predict(x_test)
print(metrics.accuracy_score(test_tags, y_pred))

print("RandomForest_200****************************************************************")
clf=RandomForestClassifier(n_estimators=200)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train, train_tags)
y_pred=clf.predict(x_test)
print(metrics.accuracy_score(test_tags, y_pred))

from xgboost.sklearn import XGBClassifier 
print("XGBClassifier****************************************************************")
clf=XGBClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train, train_tags)
y_pred=clf.predict(x_test)
print(metrics.accuracy_score(test_tags, y_pred))

print("BaggingClassifier****************************************************************")
from sklearn.ensemble.bagging import BaggingClassifier
clf=BaggingClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train, train_tags)
y_pred=clf.predict(x_test)
print(metrics.accuracy_score(test_tags, y_pred))


print("AdaBoostClassifier****************************************************************")
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
clf=AdaBoostClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train, train_tags)
y_pred=clf.predict(x_test)
print(metrics.accuracy_score(test_tags, y_pred))




from keras.models import *
from keras.layers import *

model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))#512 neurons in the first hidden layer
model.add(Activation('relu'))
#model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
#model.add(Dropout(0.3))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()
 
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=30,
                    verbose=1,
                    validation_split=0.1)



#Evaluate model.............................
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
 
print('Test accuracy:', score[1])


In [5]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv"]

#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


#X_train = np.nan_to_num(X_train)
#X_test = np.nan_to_num(X_test)


print("****************************************************************")


#..................................................................................

data_tags = ["type","gender","tweet"]
train_data_list = []
i=0
for f in y_train:
   
    train_data_list.append((f, y_train[i],all_training_text[i]))
    i += 1
# We have training data available as dictionary filename, category, data
train_data = pd.DataFrame.from_records(train_data_list, columns=data_tags)




#..................................................................................

test_data_list = []
ii=0
for f in y_test:
   
    test_data_list.append((f, y_test[ii], all_test_text[ii]))
    ii += 1
# We have training data available as dictionary filename, category, data
test_data = pd.DataFrame.from_records(test_data_list, columns=data_tags)

#..................................................................................


import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Dense, Conv1D, Flatten
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import Activation, Conv1D, Dense, Embedding, Flatten, Input, MaxPooling1D, Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout



# :Tokenize and Prepare Vocabulary

train_posts = train_data['tweet']
train_tags = train_data['type']
train_tags=array([1 if x=="M" else 0 for x in train_tags])
#train_tags=train_tags.reshape(-1, 1)
#print("train_tags", print(train_tags.shape), (train_tags))

test_posts = test_data['tweet']
test_tags = test_data['type']
test_tags=[1 if x=="M" else 0 for x in test_tags]




# define train documents
docs = train_posts

# define train class labels
labels = train_tags

# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
print("vocab_size.....", vocab_size)


# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
#print(encoded_docs)

# pad documents to a max length of 4 words
max_length = 1000

padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

#print(padded_docs)



# define test documents
test_docs = test_posts

# define train class labels
test_labels = test_tags

# prepare tokenizer
test_t = Tokenizer()
test_t.fit_on_texts(test_docs)
test_vocab_size = len(test_t.word_index) + 1

# integer encode the documents
test_encoded_docs = test_t.texts_to_sequences(test_docs)
#print(encoded_docs)

# pad documents to a max length of 4 words

test_padded_docs = pad_sequences(test_encoded_docs, maxlen=max_length, padding='post')
#print(padded_docs)


# load the whole embedding into memory
embeddings_index = dict()
f = open('/Users/catherine/Downloads/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        
        
print("vocab_size, (len(embedding_matrix))",vocab_size, (len(embedding_matrix) ) ) 
      
      
# define model_1
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length, trainable= True))
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())
# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(padded_docs, labels, epochs=5, verbose=2)
# evaluate
loss, acc = model.evaluate(test_padded_docs, test_labels, verbose=0)
print('model_1 Accuracy: %f' % (acc*100))
        
# define model_2
model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable= True))
#model.add(Embedding(vocab_size, 100, input_length=max_length, trainable= False))
model.add(Conv1D(50, 5))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(padded_docs.shape)
model.fit(padded_docs, labels, epochs=5, batch_size=64)

loss, accuracy = model.evaluate(test_padded_docs, test_labels, verbose=0)
print('model_2 Accuracy: %f' % (accuracy*100))

# create the model_3

# create the model
embedding_vecor_length = 32
model = Sequential()
#model.add(Embedding(top_words, len(embedding_matrix), input_length=max_length))
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable= False))
model.add(Dropout(0.2))
model.add(LSTM(400))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(padded_docs, labels, epochs=5, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(test_padded_docs, test_labels, verbose=0)
print("model_3 Accuracy: %.2f%%" % (scores[1]*100))




# create the model_4
model = Sequential()
model.add(Embedding(len(embedding_matrix), 100, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(Bidirectional(LSTM(128)))
#model.add(Bidirectional(LSTM(128, dropout=0.2)))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(1, activation='softmax'))
# Adam Optimiser
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(padded_docs, labels, epochs=5, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(test_padded_docs, test_labels, verbose=0)
print('model_4 Accuracy: %f' % (scores[1]*100))


# create the model_5
model = Sequential()
model.add(Embedding(len(embedding_matrix), 100, weights=[embedding_matrix], input_length=max_length, trainable=False))
#model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
#model.add(SpatialDropout1D(0.2))
#model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(400))
model.add(Dense(1, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(padded_docs, labels, epochs=5, batch_size=64)
scores = model.evaluate(test_padded_docs, test_labels, verbose=0)
print('model_5 Accuracy: %f' % (scores[1]*100))



txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv 1063
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv 1109
length of features used = 2172
len(all_training_text) 152
len(X_train)= 152 len(y_train)= 152
len(all_test_text) 142
len(X_test= 142 len(y_test)= 142
****************************************************************
vocab_size..... 18374
Loaded 400000 word vectors.
vocab_size, (len(embedding_matrix)) 18374 18374
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 1000, 100)         1837400   
_________________

In [99]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv"]

#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
#print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        #print(txt_file, len(word_list1))
        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
#X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
#print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
#X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
#print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


#X_train = np.nan_to_num(X_train)
#X_test = np.nan_to_num(X_test)


print("****************************************************************")


#..................................................................................

data_tags = ["type","gender","tweet"]
train_data_list = []
i=0
for f in y_train:
   
    train_data_list.append((f, y_train[i],all_training_text[i]))
    i += 1
# We have training data available as dictionary filename, category, data
train_data = pd.DataFrame.from_records(train_data_list, columns=data_tags)




#..................................................................................

test_data_list = []
ii=0
for f in y_test:
   
    test_data_list.append((f, y_test[ii], all_test_text[ii]))
    ii += 1
# We have training data available as dictionary filename, category, data
test_data = pd.DataFrame.from_records(test_data_list, columns=data_tags)

#..................................................................................


import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path
from numpy import array
from numpy import asarray
from numpy import zeros
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Dense, Conv1D, Flatten
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import Activation, Conv1D, Dense, Embedding, Flatten, Input, MaxPooling1D, Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.metrics import categorical_accuracy
from keras import layers
from keras.regularizers import L1L2
from sklearn.model_selection import StratifiedKFold



# :Tokenize and Prepare Vocabulary

train_posts = train_data['tweet']
train_tags = train_data['type']
train_tags=array([1 if x=="M" else 0 for x in train_tags])
#train_tags=train_tags.reshape(-1, 1)
#print("train_tags", print(train_tags.shape), (train_tags))

test_posts = test_data['tweet']
test_tags = test_data['type']
test_tags=[1 if x=="M" else 0 for x in test_tags]




# define train documents
docs = train_posts

# define train class labels
from keras.utils import to_categorical
train_labels = to_categorical(train_tags)
print("train_labels.shape...",train_labels.shape)




vocab_size = 50000
vocab_size =len(word_index)

tokenizer = Tokenizer(num_words=vocab_size) # Setup tokenizer
tokenizer.fit_on_texts(docs)
train_sequences = tokenizer.texts_to_sequences(docs) # Generate sequences
print("length_train_sequences.....", len(train_sequences[0]), len(train_sequences[10]), len(train_sequences[20]))

# pad documents to a max length of 1000 words
max_length = 2500
train_data = pad_sequences(train_sequences, maxlen=max_length)
print("length_train_data.shape.....", len(train_data[0]))


# define test documents
test_docs = test_posts

# define train class labels
#test_labels = test_tags
test_labels = to_categorical(np.asarray(test_tags))
print("test_labels.shape....",test_labels.shape)



# integer encode the documents
#test_encoded_docs = test_t.texts_to_sequences(test_docs)
tokenizer = Tokenizer(num_words=vocab_size) # Setup test tokenizer
tokenizer.fit_on_texts(test_docs)
test_sequences = tokenizer.texts_to_sequences(test_docs) # Generate sequences
test_data = pad_sequences(test_sequences, maxlen=max_length)





# load the whole embedding into memory
#embeddings_index = dict()
#glove_dir  = open('/Users/catherine/Downloads/glove.6B.100d.txt')
glove_dir  = '/Users/catherine/Downloads'

embeddings_index = {} # We create a dictionary of word -> embedding

with open(os.path.join(glove_dir, 'glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0] # The first value is the word, the rest are the values of the embedding
        embedding = np.asarray(values[1:], dtype='float32') # Load embedding
        embeddings_index[word] = embedding # Add embedding to our embedding dictionary

print('Found {:,} word vectors in GloVe.'.format(len(embeddings_index)))

        
        
embedding_dim = 100 # We use 100 dimensional glove vectors

word_index = tokenizer.word_index
nb_words = min(vocab_size, len(word_index)) # How many words are there actually
print("nb_words...... ",nb_words )
nb_words = vocab_size
nb_words = len(word_index)
embedding_matrix = np.zeros((nb_words, embedding_dim))

# The vectors need to be in the same position as their index. 
# Meaning a word with token 1 needs to be in the second row (rows start with zero) and so on

# Loop over all words in the word index
for word, i in word_index.items():
    # If we are above the amount of words we want to use we do nothing
    if i >= vocab_size: 
        continue
    # Get the embedding vector for the word
    embedding_vector = embeddings_index.get(word)
    # If there is an embedding vector, put it in the embedding matrix
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector
        

        
 


#trainable=False... Accuracy: 61.267606
model = Sequential()
model.add(Embedding(vocab_size, 
                    embedding_dim, 
                    input_length=max_length, 
                    weights = [embedding_matrix], 
                    trainable=False))
model.add(Conv1D(512, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Dropout(0.1))
model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[categorical_accuracy])
#model.fit(train_data, train_labels, epochs=10, validation_split=0.0)
model.fit(train_data, train_labels,  validation_data=(test_data, test_labels), epochs=10)
print("model.metrics_names", model.metrics_names)
scores = model.evaluate(test_data, test_labels, verbose=0)
print('model_6 Accuracy: %f' % (scores[1]*100), scores)


model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=max_length, 
                           trainable=True))
model.add(layers.GlobalMaxPool1D())
#model.add(Flatten())
model.add(Dense(units=100, activation='relu'  ))
model.add(Dropout(0.3))
#model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(2, activation='sigmoid'))
#model.compile(optimizer='adam',
              #loss='binary_crossentropy',
              #metrics=['accuracy'])
model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print("model.metrics_names", model.metrics_names)

#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[categorical_accuracy])
model.fit(train_data, train_labels,  validation_data=(test_data, test_labels), epochs=10)
scores = model.evaluate(test_data, test_labels, verbose=0)
print('model_7 Accuracy: %f' % (scores[1]*100), scores)



len(all_training_text) 152
len(all_test_text) 142
****************************************************************
train_labels.shape... (152, 2)
length_train_sequences..... 2360 1213 1522
length_train_data.shape..... 2500
test_labels.shape.... (142, 2)
Found 400,000 word vectors in GloVe.
nb_words......  17423


NameError: name 'rate' is not defined

In [ ]:
check the validation error during each epoch......
fully connected layers works for out put that have got shapes but not for only labels...
convolutional layer best layer when input structure is important

##Create a bigger model
bigger_model = keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(NUM_WORDS,)),
    keras.layers.Dense(512, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])


In [ ]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []
for train, test in kfold.split(X, Y):
    
    model = Sequential()
    model.add(Embedding(vocab_size, 
                    embedding_dim, 
                    input_length=max_length, 
                    weights = [embedding_matrix], 
                    trainable=False))
    model.add(Conv1D(512, 3, activation='relu'))
    model.add(MaxPooling1D(3))
    model.add(Conv1D(128, 3, activation='relu'))
    model.add(MaxPooling1D(3))
    model.add(Conv1D(128, 3, activation='relu'))
    model.add(MaxPooling1D(3))
    model.add(Conv1D(128, 3, activation='relu'))
    model.add(MaxPooling1D(3))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.summary()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[categorical_accuracy])
    model.fit(train_data, train_labels,  epochs=10, batch_size=10)
    scores = model.evaluate(test_data, test_labels, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))

In [2]:
#import sys
#sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv"]

#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
#print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        #print(txt_file, len(word_list1))
        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
#X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
#print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
#X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
#print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


#X_train = np.nan_to_num(X_train)
#X_test = np.nan_to_num(X_test)


print("****************************************************************")


#..................................................................................

data_tags = ["type","gender","tweet"]
train_data_list = []
i=0
for f in y_train:
   
    train_data_list.append((f, y_train[i],all_training_text[i]))
    i += 1
# We have training data available as dictionary filename, category, data
train_data = pd.DataFrame.from_records(train_data_list, columns=data_tags)




#..................................................................................

test_data_list = []
ii=0
for f in y_test:
   
    test_data_list.append((f, y_test[ii], all_test_text[ii]))
    ii += 1
# We have training data available as dictionary filename, category, data
test_data = pd.DataFrame.from_records(test_data_list, columns=data_tags)

#..................................................................................


import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path
from numpy import array
from numpy import asarray
from numpy import zeros
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Dense, Conv1D, Flatten
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import Activation, Conv1D, Dense, Embedding, Flatten, Input, MaxPooling1D, Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.metrics import categorical_accuracy
from keras import layers
from keras.regularizers import L1L2
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)



# :Tokenize and Prepare Vocabulary

train_posts = train_data['tweet']
train_tags = train_data['type']
train_tags=array([1 if x=="M" else 0 for x in train_tags])
#train_tags=train_tags.reshape(-1, 1)
#print("train_tags", print(train_tags.shape), (train_tags))

test_posts = test_data['tweet']
test_tags = test_data['type']
test_tags=[1 if x=="M" else 0 for x in test_tags]




# define train documents
docs = train_posts

# define train class labels

le = LabelEncoder()
Y = le.fit_transform(train_tags)
train_labels = Y.reshape(-1,1)
print(train_labels.shape)



max_words = 1000
max_len = 1500

tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(docs)
train_sequences = tok.texts_to_sequences(docs)
train_sequences_matrix = sequence.pad_sequences(train_sequences,maxlen=max_len)
print("train_sequences_matrix",len(train_sequences_matrix), train_sequences_matrix[0, :])



# define test documents
test_docs = test_posts

# define train class labels
#test_labels = test_tags
la = LabelEncoder()
Yy = la.fit_transform(test_tags)
test_labels = Yy.reshape(-1,1)
print(test_labels.shape)

# integer encode the documents
#test_encoded_docs = test_t.texts_to_sequences(test_docs)

tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(test_docs)
test_sequences = tok.texts_to_sequences(test_docs)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

X = np.concatenate((train_sequences_matrix, test_sequences_matrix))
Y= np.concatenate((train_labels, test_labels))

#Train test split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print("X_train", X_train.shape,Y_train.shape)
print("X_test", X_test.shape,Y_test.shape)

def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(256)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
model.fit(X_train,Y_train,batch_size=128,epochs=10)
#model.fit(train_sequences_matrix,train_labels,batch_size=128,epochs=10)#,
          #validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])


accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

len(all_training_text) 152
len(all_test_text) 142
****************************************************************
(152, 1)
train_sequences_matrix 152 [ 28  36  30 ... 420  16  24]
(142, 1)
X_train (264, 1500) (264, 1)
X_test (30, 1500) (30, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1500)              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 1500, 50)          50000     
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               314368    
_________________________________________________________________
FC1 (Dense)                  (None, 256)               65792     
_________________________________________________________________
activation_3 (Activation)    (None, 256)               0         
_____________________________

In [12]:
#import sys
#sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
#word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv"]

#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
#print("txt_files of features used",txt_files)
#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        reader = csv.reader(f, delimiter=",")
#        next(reader) # skip header
#        word_list1 =  [r[0] for r in reader]
        #print(txt_file, len(word_list1))
#        word_list = word_list + word_list1
#print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
#X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
#print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
#X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
#print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


#X_train = np.nan_to_num(X_train)
#X_test = np.nan_to_num(X_test)


print("****************************************************************")


#..................................................................................

data_tags = ["type","gender","tweet"]
train_data_list = []
i=0
for f in y_train:
   
    train_data_list.append((f, y_train[i],all_training_text[i]))
    i += 1
# We have training data available as dictionary filename, category, data
train_data = pd.DataFrame.from_records(train_data_list, columns=data_tags)




#..................................................................................

test_data_list = []
ii=0
for f in y_test:
   
    test_data_list.append((f, y_test[ii], all_test_text[ii]))
    ii += 1
# We have training data available as dictionary filename, category, data
test_data = pd.DataFrame.from_records(test_data_list, columns=data_tags)

#..................................................................................


import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path
from numpy import array
from numpy import asarray
from numpy import zeros
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Dense, Conv1D, Flatten
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import Activation, Conv1D, Dense, Embedding, Flatten, Input, MaxPooling1D, Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.metrics import categorical_accuracy
from keras import layers
from keras.regularizers import L1L2
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)



# :Tokenize and Prepare Vocabulary

train_posts = train_data['tweet']
train_tags = train_data['type']
train_tags=array([1 if x=="M" else 0 for x in train_tags])
#train_tags=train_tags.reshape(-1, 1)
#print("train_tags", print(train_tags.shape), (train_tags))

test_posts = test_data['tweet']
test_tags = test_data['type']
test_tags=[1 if x=="M" else 0 for x in test_tags]




from string import punctuation
from os import listdir
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D





# load the vocabulary
# vocab_filename = ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_vocubulary.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_ vocubulary_set.1.csv']
vocab_filename = ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv']

for txt_file in vocab_filename:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        #print(txt_file, len(word_list1))
        word_list = word_list + word_list1
vocab = set(word_list)
print("len__vocab",len(vocab))

# load all training reviews
#positive_docs = process_docs('/Users/catherine/Downloads/review_polarity/txt_sentoken/pos', vocab, True)
#negative_docs = process_docs('/Users/catherine/Downloads/review_polarity/txt_sentoken/neg', vocab, True)
#train_docs = negative_docs + positive_docs
train_docs=train_posts

train_tokens=[x  for z in train_docs for x  in z]
fdist=nltk.FreqDist(train_tokens)
fdist = sorted(dict(fdist).items(), key=operator.itemgetter(1), reverse=True)
#print(fdist)
print("len(train_tokens),len(fdist),fdist[0]",len(train_tokens),len(fdist),fdist[0])
# keep tokens with a min occurrence
min_occurane = 9
tokens = [k for k,c in dict(fdist).items() if c >= min_occurane]
print("len(tokens)for vocab",len(tokens))

# save list to file
def save_list(lines, filename):
	# convert lines to a single blob of text
	data = '\n'.join(lines)
	# open file
	file = open(filename, 'w')
	# write text
	file.write(data)
	# close file
	file.close()

# save tokens to a vocabulary file
save_list(tokens, 'vocab.txt')



# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# load the vocabulary
# vocab_filename = 'vocab.txt'
# vocab = load_doc(vocab_filename)
# vocab = vocab.split()
# vocab = set(vocab)


# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
train_docs= [[w for w in tokens if w in vocab ] for tokens in train_docs]
#[[number+1 for number in group] for group in x]
tokenizer.fit_on_texts(train_docs)


# sequence encode
encoded_docs = tokenizer.texts_to_sequences(train_docs)

# pad sequences
max_length = max([len(s) for s in train_docs])
Xtrain = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

# define training labels
ytrain = array([0 for _ in range(900)] + [1 for _ in range(900)])
ytrain = train_tags

test_docs = test_posts
#test_docs = [w for w in tokens for tokens in test_docs if w in vocab ]
test_docs= [[w for w in tokens if w in vocab ] for tokens in test_docs]
# sequence encode
encoded_docs = tokenizer.texts_to_sequences(test_docs)
# pad sequences
Xtest = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
# define test labels
ytest = array([0 for _ in range(100)] + [1 for _ in range(100)])
ytest = test_tags

# define vocabulary size (largest integer value)
vocab_size = len(tokenizer.word_index) + 1
print("vocab_size", vocab_size)

# load embedding from file
raw_embedding = load_embedding('/Users/catherine/Downloads/glove.6B.100d.txt')
# get vectors in the right order
embedding_vectors = get_weight_matrix(raw_embedding, tokenizer.word_index)
# create the embedding layer
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_vectors], input_length=max_length, trainable=False)

# define model1
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
print(model.summary())
# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, ytrain, epochs=10, verbose=2)
# evaluate
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))

# define model2.... HAS NO WEIGHTS
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit network
model.fit(Xtrain, ytrain, epochs=10, verbose=2)

# evaluate
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))

# define model3
#embedding_layer = Embedding(vocab_size, 100, weights=[embedding_vectors], input_length=max_length, trainable=False)
def RNN():
    inputs = Input(name='inputs',shape=[max_length])
    layer = Embedding(vocab_size,100,input_length=max_length)(inputs)
    layer = LSTM(256)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
model.fit(Xtrain, ytrain,batch_size=128,epochs=10)
#model.fit(train_sequences_matrix,train_labels,batch_size=128,epochs=10)#,
          #validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

accr = model.evaluate(Xtest, ytest)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))


# define model4
#trainable=False... Accuracy: 61.267606
#embedding_layer = Embedding(vocab_size, 100, weights=[embedding_vectors], input_length=max_length, trainable=False)
model = Sequential()
model.add(Embedding(vocab_size, 
                    embedding_dim, 
                    input_length=max_length, 
                    weights = [embedding_vectors], 
                    trainable=False))
model.add(Conv1D(512, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Dropout(0.1))
model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[categorical_accuracy])
#model.fit(train_data, train_labels, epochs=10, validation_split=0.0)
model.fit(Xtrain, ytrain,  validation_data=(Xtest, ytest), epochs=10)
print("model.metrics_names", model.metrics_names)
scores = model.evaluate(Xtest, ytest, verbose=0)
print('model_4 Accuracy: %f' % (scores[1]*100), scores)

                        

# define model_1
#embedding_layer = Embedding(vocab_size, 100, weights=[embedding_vectors], input_length=max_length, trainable=False)
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length, trainable= True))
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())
# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, ytrain, epochs=10, verbose=2)
# evaluate
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('model_5 Accuracy: %f' % (acc*100))
        
# define model_2
#embedding_layer = Embedding(vocab_size, 100, weights=[embedding_vectors], input_length=max_length, trainable=False)
model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_vectors], input_length=max_length, trainable= True))
#model.add(Embedding(vocab_size, 100, input_length=max_length, trainable= False))
model.add(Conv1D(50, 5))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(padded_docs.shape)
model.fit(Xtrain, ytrain, epochs=10, batch_size=64)

loss, accuracy = model.evaluate(Xtest, ytest, verbose=0)
print('model_6 Accuracy: %f' % (accuracy*100))

# create the model_3

# create the model
#embedding_layer = Embedding(vocab_size, 100, weights=[embedding_vectors], input_length=max_length, trainable=False)
embedding_vecor_length = 32
model = Sequential()
#model.add(Embedding(top_words, len(embedding_matrix), input_length=max_length))
model.add(Embedding(vocab_size, 100, weights=[embedding_vectors], input_length=max_length, trainable= False))
model.add(Dropout(0.2))
model.add(LSTM(400))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(Xtrain, ytrain, epochs=10, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(Xtest, ytest, verbose=0)
print("model_7 Accuracy: %.2f%%" % (scores[1]*100))




# create the model_4
model = Sequential()
model.add(Embedding(len(embedding_vectors), 100, weights=[embedding_vectors], input_length=max_length, trainable=False))
model.add(Bidirectional(LSTM(128)))
#model.add(Bidirectional(LSTM(128, dropout=0.2)))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(1, activation='softmax'))
# Adam Optimiser
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(Xtrain, ytrain, epochs=10, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(Xtest, ytest, verbose=0)
print('model_8 Accuracy: %f' % (scores[1]*100))


# create the model_5
model = Sequential()
model.add(Embedding(len(embedding_vectors), 100, weights=[embedding_vectors], input_length=max_length, trainable=False))
#model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
#model.add(SpatialDropout1D(0.2))
#model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(400))
model.add(Dense(1, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(Xtrain, ytrain, epochs=10, batch_size=64)
scores = model.evaluate(Xtest, ytest, verbose=0)
print('model_9 Accuracy: %f' % (scores[1]*100))




len(all_training_text) 152
len(all_test_text) 142
****************************************************************
len__vocab 2172
len(train_tokens),len(fdist),fdist[0] 216597 18373 ('username', 8321)
len(tokens)for vocab 1964
vocab_size 2173


NameError: name 'load_embedding' is not defined

In [ ]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_train.1.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_train.1.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_male_test.1.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles//PAN2015_tweet_female_test.1.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
#txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015ProbD.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2015/PAN2015.1_textfiles/PAN2015_tweet_male_chosen_set.1.csv"]

#txt_files =["/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2015/output2015GSS.txt"]
print("txt_files of features used",txt_files)
for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))

#for txt_file in txt_files:
#    with open(txt_file, mode="r", encoding="utf-8") as f:
#        word_list =  ([(r.split(":"))[1] for r in f])[0:200]
#        print(txt_file, len(word_list))
#        #word_list = word_list + word_list1
#print("length of features used =", len(word_list))

"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
all_training_text = ( [((all_txt(M_train_file,  F_train_file))[i][1])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
print("len(all_training_text)",len(all_training_text))
#X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [((all_txt(M_train_file,  F_train_file))[i][0])  for i in range(len((all_txt(M_train_file,  F_train_file))))])
#print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

all_test_text = ( [((all_txt(M_test_file,  F_test_file))[i][1])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
print("len(all_test_text)",len(all_test_text))
#X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [((all_txt(M_test_file,  F_test_file))[i][0])  for i in range(len((all_txt(M_test_file,  F_test_file))))])
#print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)


print("************************************ ****************************")


k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
from fuzzywuzzy import fuzz
for i in range(len(all_training_text)):
    for j in range(len(all_test_text)):
        print(fuzz.ratio(all_training_text[i], all_test_text[j]))
        